In [1]:
import typing

import pandas as pd

import config
import src
import requests
import os
from dotenv import load_dotenv
load_dotenv("sjoerdAzure.env")  # Load environment variables from .env file
import time
import tqdm
#from tqdm import tqdm          #run these lines to run the ADA progress bar
#tqdm.pandas()
import json
import numpy as np
import logging
import re
#import cltrier_lib as lib

import yaml
pd.set_option('display.max_colwidth', 100) 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\sstolwi\Git

In [2]:
#set up helper variables and functions:
CFG = config.Config()

def load_json(path: str):
    with open(path, encoding='utf-8') as fp:
        return json.load(fp)
    
#set option variables:

#set options to low temperature (0,1):
options_low_str = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low_str)

MODELsmall: str = 'llama3.1:8b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELlarge: str = 'llama3.1:70b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELgpt4o = "nf-gpt-4o-2024-08-06" # in principe is er nu van elk model een nf (no filter) en een normale versie beschikbaar, de no filter versies zijn alleen voor onderzoekers beschikbaar voor analyze van content die niet door de filter heen zou komen.
MODELgpt4T = "nf-gpt-4-turbo" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.
MODELgpt4 = "nf-gpt-4" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.

options_zero_str = """
seed: 42
temperature: 0
"""
options_zero = yaml.safe_load(options_zero_str)

temperature_0 : int = 0
SEED: int = 42
MAX10: int = 10
TOPP1: int = 1


options_large_str = """
seed: 42
temperature: 0
num_predict: 2000
"""
options_large = yaml.safe_load(options_large_str)

#load environment variables:
api_key = os.environ.get('sjoerd_key')

#setttings:
api_endpoint = "https://ai-research-proxy.azurewebsites.net/chat/completions"
api_endpoint_embed = "https://ai-research-proxy.azurewebsites.net/embeddings"
####### API REQUEST FORMATTING ######
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + api_key
}

In [3]:
def get_ada_embedding(text):
    retry_count = 0
    max_retries = 10
    while retry_count < max_retries:
        try:
            response = requests.post(
                url=api_endpoint_embed,
                headers=headers,
                json={
                    "model": "text-embedding-ada-002",
                    "input": text,
                }
            )
            if response.status_code == 200:
                return response.json()['data'][0]['embedding']
            elif response.status_code == 429:
                retry_count += 1
                wait_time = 1 + (3 * retry_count * retry_count)
                print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                print(response.text)
                time.sleep(wait_time)
            elif response.status_code == 500:
                retry_count += 1
                wait_time = 20
                print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                print(response.text)
                time.sleep(wait_time)
            else:
                print(f"Failed to connect to API. Status code: {response.status_code}")
                print(response.text)
                break
        except requests.exceptions.RequestException as e:   
            print(f"Failed to connect to API: {e}")
            retry_count += 1
            wait_time = 60
            print(f"Retrying in {wait_time} seconds...")
            time.sleep(wait_time)        

def convert_to_int(value):
    if isinstance(value, list):
        return np.nan
    try:
        return int(value)
    except ValueError:
        return np.nan
      


In [ ]:
dataset: pd.DataFrame = pd.read_csv('data/publicsphere/full_data.csv')
dataset_claim_embeds = pd.read_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')


In [5]:
# Parse datetime values
dataset['parsed_datetime'] = pd.to_datetime(dataset['Time_comment'], errors='coerce', utc=True)

# Fill NaT values for date-only entries
dataset['parsed_datetime'] = dataset['parsed_datetime'].fillna(pd.to_datetime(dataset['Time_comment'], format='%Y-%m-%d', errors='coerce', utc=True))


print(dataset.loc[:, ['Time_comment', 'parsed_datetime']].head(5))

dataset.loc[:, 'dataset_index'] = dataset.index
YT_input = dataset[dataset['Platform'] == 1]
X_input = dataset[dataset['Platform'] == 2]

               Time_comment           parsed_datetime
0  2017-11-16T09:14:09.000Z 2017-11-16 09:14:09+00:00
1  2019-08-12T20:22:06.000Z 2019-08-12 20:22:06+00:00
2                2019-03-26 2019-03-26 00:00:00+00:00
3  2018-10-03T05:44:24.000Z 2018-10-03 05:44:24+00:00
4  2018-08-28T05:38:51.000Z 2018-08-28 05:38:51+00:00


In [4]:
#showname and videoTitle make most sense to group by:
print(dataset_claim_embeds['showName'].value_counts())
print(dataset_claim_embeds['videoTitle'].value_counts())

showName
NBC News                                201
The Daily Show with Trevor Noah         188
The Late Show with Stephen Colbert      186
Hardball with Chris Matthews            177
Real Time with Bill Maher               172
Hannity                                 171
Tucker Carlson Tonight                  155
Fox News                                152
Late Night with Seth Meyers             148
Face the Nation                         128
60 Minutes                              125
CBS Evening News                        116
ABC News                                109
LastWeekTonight                         106
The 11th Hour                           101
CNN                                      94
Anderson Cooper 360                      89
TuckerCarlson                            82
hardball                                 81
AC360                                    79
Hardball with Chris Matthews (MSNBC)     59
Tucker Carlson Tonight (FoxNews)         58
Full Frontal with Saman

In [6]:
#select the rows in the dataset that have the a videoTitle with more than 20 comments:   
selectedvideos = dataset_claim_embeds[dataset_claim_embeds['videoTitle'].isin(dataset_comment_embeds['videoTitle'].value_counts()[dataset_comment_embeds['videoTitle'].value_counts() >= 20].index)]

In [6]:
SYSTEM_claim: str = \
    """
        Instruction:

        You are a text annotation assistant. Analyze a social media comment enclosed in chevrons <..>. Identify and list the claims within this comment. Claims can be related to events, issues, opinions or concerns.
        Claims are defined as the main assertion or conclusion of an argument.
        You summarize each claim into a short simple sentence.

        Response format:

        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].

        Response format template:
        
        ["claim 1", "claim 2", ... "claim x"]
	"""

#note updated on 30-12-2024:
SYSTEM_expansion: str = \
    """
        # Instruction\n\nRephrase a social media post to reflect its meaning within the context of a conversation thread:\n\n1. You'll receive a thread in chevrons `<...>` and a target post in double chevrons `<<...>>`.\n2. If the post is clear without context, repeat it as-is.\n3. If the post depends on context, expand it to include necessary details.\n4. Respond with only the expanded post. \n5. If the post does not refer to context provided in the thread, or if in doubt, respond with the exact target post as you received it. If no target post was presented reply with an empty list [] \n\n## Example\n\n**Input**:\n- Thread: <'Comment 1', 'Comment 2', 'Comment 3'>\n- Target reply: `<<This is so relatable!>>`\n\n**Output**:\n- '[context from previous comments] is so relatable!'\n\n**Text:
    """

SYSTEM_claim_new: str = \
    """
        # Instruction

        Identify whether a claim adds new information (2), expands on information (1) or does not add new information (0) with respect to preceding claims. New information is present (2) if the claim introduces a new argument or perspective. 
        A claim expands on an existing argument or perspective (1) if it strongly related to it, or it provides additional context or examples to an argument or perspective in a preceding claim, or if it provides a sub-argument (parent-child relation). 
        If the claim is very similar to a preceding claim or does not add any perspective or argument, assign a value of 0. Follow these steps:


        Follow these steps:

        1. You will receive a target claim in double chevrons <<...>> along with the preceding claims in single chevrons <...>.
        2. Identify if new information is presented in the target claim with respect to the preceding posts. 
        4. If in doubt, assign a value of 0.

        Respond with only the predicted class (0 or 1 or 2) of the request. Do not include any additional text or explanations.
        Class:
	"""

SYSTEM_claim_simscore: str = \
    """
        # Instruction

        Identify degree of similarity of the information presented in a claim with respect to preceding claims in a thread on an integer scale of 1 (completely dissimilar) to 10 (identical). A score of 5 indicates that the claims share context or topic, but otherwise present different information. 
        Follow these steps:

        1. You will receive a target claim in double chevrons <<...>> along with a JSON containing the preceding claims and their claim_index enclosed in single chevrons <...>.
        2. Determine the most similar claim to the target claim in the preceding claims in terms of the information they present.. If only one preceding claim is provided pick this claim as the most similar claim.
        3. Find the claim_index of this most similar claim. If you can't decide which claim is most similar, pick a preceding comment at random.
        4. Identify the degree of similarity of the target claim with respect to that claim on a range of 1-10. 
        5. If no target claim is porvided or only '[]' return an empty list [] as value for both the most_similar_claim_index and the similarity_score, if you can't decide on the similarity score return an empty list for that value. If no preceding claims are provided or only an empty string '', return an empty list [] as value for both the most_similar_claim_index and the similarity_score.
        6. Always and only respond with the claim_index and similarity score.

        Response format in JSON:

        [
            {
                "most_similar_claim_index": "1",
                "similarity_score": "1"	
            }
        ]
	"""

SYSTEM_post_simscore: str = \
    """
        # Instruction 
        
        Identify degree of similarity of the information presented in a social media comment with respect to preceding comments in a thread on an integer scale of 1 (completely dissimilar) to 10 (identical). A score of 5 indicates that the comments share context or topic, but otherwise present different information. 
        Follow these steps:
        
        1. You will receive a target comment in double chevrons <<...>> along with a JSON containing the preceding comments and their comment_index enclosed in single chevrons <...>.
        2. Determine the most similar comment to the target comment in the preceding comments in terms of the information they present. If only one preceding comment is provided pick this comment as the most similar comment.
        3. Find the comment_index of this most similar comment. If you can't decide which comment is most similar, pick a preceding comment at random.
        4. Identify the degree of similarity of the target comment with respect to that comment on a range of 1-10. 
        5. If no target comment is provided or only '[]' return an empty list [] as value for both the most_similar_comment_index and the similarity_score, if you can't decide on the similarity score return an empty list for that value.
        6. Always and only respond with the comment_index and similarity score.
        
        Response format in JSON:
        
        [
            {
                "most_similar_comment_index": "1",
                "similarity_score": "1"	
            }
        ]
	"""

In [6]:
#find out which posts are political, and which are not:
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset_claim_embeds["commentText"], MODELsmall, options=options_zero)
    )
    for label, path in CFG.prompt_classify_files.items() if label in ['political_post']
}

classifying political_post: 100%|██████████| 3862/3862 [17:10<00:00,  3.75it/s]


In [7]:
#join to the dataset:
for label, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to include the label
    preds_df.columns = [f'Llama31_{label}_8b']
    dataset_claim_embeds = dataset_claim_embeds.join(preds_df)

political_post
political        2546
non-political    1155
Name: count, dtype: int64
------------------------------------------


In [8]:
dataset_claim_embeds.loc[:, 'Llama31_political_fill_8b'] = dataset_claim_embeds.loc[:, 'Llama31_political_post_8b'].fillna('non-political')
dataset_claim_embeds.loc[:, 'Llama31_political_fill_8b_score'] = dataset_claim_embeds.loc[:, 'Llama31_political_post_8b'].map({'non-political': 0, 'political': 1}).fillna(0).astype(int)

In [10]:
#save data:
dataset_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')

In [7]:
#load data:
dataset_claim_embeds = pd.read_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')

In [12]:
dataset_claim_embeds.columns.to_list()

['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'commentText',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',


In [8]:
#find threads with most political posts:
#select the rows in the dataset that are in a thread with more political posts than cutoff:   
# Define the cutoff value
cutoff_value = 8

# Group by 'thread_title' and count the number of political posts
thread_counts = dataset_claim_embeds.groupby('videoTitle')['Llama31_political_fill_8b_score'].sum()

# Filter the threads based on the cutoff value
pol_threads = thread_counts[thread_counts > cutoff_value].index

# Select the posts in the filtered threads
pol_posts = dataset_claim_embeds[dataset_claim_embeds['videoTitle'].isin(pol_threads)]

# Display the filtered posts
print(pol_posts.loc[pol_posts['Llama31_political_fill_8b_score'] == 1, 'commentText'].to_list())

["That's a vicious insult!!! What did a box of rocks ever do to you that you would slander it like that? I represent the coalition for mineral rights. Minerals have rights to.", 'I agree.  Maybe he was sending a message to his mistress.', 'President Trump made 8,158 false or misleading claims in his first two years ... besides his entire life, businesses, deals, have been a lie as well  .... should he apologize?', 'NPC-90210-RockTheVote   Heâ€™s getting his oil checked!', "Gee, someone is upset that they are trying to give Fox a run for their money.  Don't worry Sierra, Fox still controls that niche.", "What are they being false about?  It's a fact that the military assets are being moved.", 'Bernie is the most consistent and most  trustworthy progressive. Feel the Bern!', "USA Freedom Patriots made the Internet. \xa0Why don't you go back to fucking donkeys.", "What I seriously don't get and leaves me shaking my head in frustration:\n the USA doesn't want Iran to have nuclear weapons -

In [6]:
pol_posts.sort_values(by='Time_comment').reset_index(drop=False).Time_comment.head(5)

0    2014-06-09T23:18:16.000Z
1    2014-06-10T00:26:36.000Z
2    2014-06-10T14:26:16.000Z
3    2014-06-12T01:40:55.000Z
4    2014-06-16T08:09:41.000Z
Name: Time_comment, dtype: object

In [ ]:
#Expand the polposts to include the thread context:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
sort_pol_posts = pol_posts.sort_values(by='Time_comment').reset_index(drop=False)
groupeddata = sort_pol_posts.loc[sort_pol_posts.Llama31_political_fill_8b == 'political',:].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            chunked_result.append(
                pd.DataFrame(
                    data=[[index, row['commentText']]],
                    columns=['index', 'post_expansion']
                )
            )
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            try: 
                chunked_result.append(
                    pd.DataFrame(
                        data=[[index,
                            requests.post(
                                'https://inf.cl.uni-trier.de/',
                                json={
                                    'model': MODELlarge,
                                    'system': SYSTEM_expansion,
                                    'prompt': f'"Thread":\n<{df["commentText"][:threadset].to_list()}>, "Target reply":<<{df["commentText"][index]}>>',
                                    'options': options_zero
                                    }).json()['response']                       
                        ]],
                        columns=['index','post_expansion']
                    )
                )
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")

expanded_posts = pd.concat(chunked_result, ignore_index=True)
print(expanded_posts)

  0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
expanded_posts = pd.concat(chunked_result, ignore_index=True)
print(expanded_posts)

In [27]:
expanded_posts.set_index(expanded_posts['index'], inplace=True)
sort_pol_posts = sort_pol_posts.join(expanded_posts['post_expansion'])

In [ ]:
sort_pol_posts.loc[10:, ['commentText', 'post_expansion', 'videoTitle']].head(5)

In [35]:
#save data:
sort_pol_posts.to_parquet(f'{CFG.report_dir}/pubsphere_pol_posts.parquet')

In [ ]:
#apply generalized claim mining to get a list of claims:
chunked_result: typing.List[pd.DataFrame] = []
for index, row in tqdm.tqdm(sort_pol_posts.iterrows()):
    try: 
        chunked_result.append(
            pd.DataFrame(
                data=[
                    requests.post(
                        'https://inf.cl.uni-trier.de/',
                        json={
                            'model': MODELsmall,
                            'system': SYSTEM_claim,
                            'prompt': f'The following set of social media posts are replies to a news- or infotainment-post. '
                                    + f'Check whether your answer strictly adheres to the specified format. \n"Posts":\n<{row["commentText"]}>',
                            'options': options_zero
                            }).json()['response']                       
                ],
                columns=['claims']
            )
        )
        
    except json.JSONDecodeError:
        print("invalid json response, skipping to next batch")

In [ ]:
claim_df = pd.concat(chunked_result, ignore_index=True)
print(claim_df)

In [9]:
#and what if we apply GPT4o to the same data:
# dataset_claim_embeds["commentText"] -> get political_post
# -> get post_expansion for political posts
# -> get claims for post_expansion
# -> find diversity in claims and posts through SYSTEM_claim_simscore
# -> get ADA embeddings for posts and claims


In [10]:
#get political posts:
#classify messages using gpt4o:
run1 = ['political_post']
  
chunked_result: typing.List[pd.DataFrame] = []

for label, path in CFG.prompt_classify_files.items():
    if label in run1: 
        template = load_json(path).get('template')
        classes = load_json(path).get('classes')
        for index, row in tqdm.tqdm(dataset_claim_embeds["commentText"].items()):
            retry_count = 0
            max_retries = 10
            
            while retry_count < max_retries:
                try: 
                    response = requests.post(
                            url=api_endpoint,
                            headers=headers,
                            json={
                                'model': MODELgpt4o,
                                'messages': [
                                    {
                                        "role": "system",
                                        "content": template
                                    },
                                    {
                                        "role": "user",
                                        "content": template.format(text=row)
                                    }
                                ],
                                'temperature': temperature_0,  
                                'seed': SEED,
                                "max_tokens": MAX10
                            }
                        )  

                    if response.status_code == 200:
                        data_response = response.json()
                        chunked_result.append(
                        pd.DataFrame(
                            data=[[index, classes.get(data_response["choices"][0]["message"]["content"], None)]],                                
                            columns=['index', label]
                            )
                        )
                        break  # Exit the retry loop on success
                    elif response.status_code == 429:
                        retry_count += 1
                        wait_time = 1 + (3 * retry_count * retry_count)
                        print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    elif response.status_code == 500:
                        retry_count += 1
                        wait_time = 20
                        print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to connect to API. Status code: {response.status_code}")
                        print(response.text)
                        break
                except requests.exceptions.RequestException as e:   
                    print(f"Failed to connect to API: {e}")
                    retry_count += 1
                    wait_time = 60
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)                 
                


classifications1 = pd.concat(chunked_result, ignore_index=True)
print(classifications1) 

3862it [37:18,  1.73it/s]

      index political_post
0         0  non-political
1         1  non-political
2         2      political
3         3  non-political
4         4  non-political
...     ...            ...
3857   3857  non-political
3858   3858  non-political
3859   3859  non-political
3860   3860  non-political
3861   3861      political

[3862 rows x 2 columns]


In [11]:
political_post_gpt4o = classifications1.loc[:, ["index", "political_post"]].dropna()
political_post_gpt4o.set_index('index', drop=True, inplace=True)
dataset_claim_embeds = dataset_claim_embeds.join(political_post_gpt4o, rsuffix='_gpt4o')

dataset_claim_embeds = dataset_claim_embeds.rename(columns={
    'political_post': 'political_post_gpt4o'
})
dataset_claim_embeds.loc[:, 'political_post_gpt4o_dum'] = dataset_claim_embeds.loc[:, 'political_post_gpt4o'].map({"political": 1, "non-political":0}).fillna(0).astype(int)


In [5]:
dataset_claim_embeds.head()

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,tfidf_embed_post,embed_MXBAI_post,cosine_similarity_post_claim_MXBAI,cosine_low_high_MXBAI,tfidf_embed_post_svd,Llama31_political_post_8b,Llama31_political_fill_8b,Llama31_political_fill_8b_score,political_post_gpt4o,political_post_gpt4o_dum
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.40566757321357727, -0.032418690621852875, -0.5345820188522339, 0.25060492753982544, 0.3237237...",0.960095,1.000000,[1.0],non-political,non-political,0,non-political,0
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.49942252039909363, -0.22240903973579407, -0.4247226417064667, 0.18189606070518494, -0.0185961...",0.957984,0.991973,[0.9999999959466253],political,political,1,non-political,0
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.6132397651672363, -0.3024018704891205, -0.6756535172462463, 0.33316466212272644, 0.2613784670...",0.967763,0.996136,[1.0000000354648362],political,political,1,political,1
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.27884048223495483, -0.3008716404438019, -0.48076391220092773, 0.6041528582572937, 0.336594045...",0.952258,0.992674,[0.9999999904984119],political,political,1,non-political,0
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5210463404655457, 0.03200243413448334, -0.37679624557495117, 0.37940096855163574, 0.320245444...",0.977981,0.992789,[1.0],non-political,non-political,0,non-political,0


In [203]:
#save data:
dataset_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')

In [8]:
#find threads with most POTENTIAL political posts:
#select the rows in the dataset that are in a thread with more political posts than cutoff:   
# Define the cutoff value
cutoff_value = 8

dataset_claim_embeds['either_political'] = dataset_claim_embeds[['Llama31_political_fill_8b_score', 'political_post_gpt4o_dum']].max(axis=1)

# Group by 'thread_title' and count the number of political posts
thread_counts = dataset_claim_embeds.groupby('videoTitle')['either_political'].sum()

# Filter the threads based on the cutoff value
pol_threads_either = thread_counts[thread_counts > cutoff_value].index

# Select the posts in the filtered threads
pol_posts_either = dataset_claim_embeds[dataset_claim_embeds['videoTitle'].isin(pol_threads_either)]

# Display the filtered posts
print(pol_posts_either.loc[pol_posts_either['either_political'] == 1, 'commentText'].to_list())

["That's a vicious insult!!! What did a box of rocks ever do to you that you would slander it like that? I represent the coalition for mineral rights. Minerals have rights to.", 'I agree.  Maybe he was sending a message to his mistress.', 'President Trump made 8,158 false or misleading claims in his first two years ... besides his entire life, businesses, deals, have been a lie as well  .... should he apologize?', 'NPC-90210-RockTheVote   Heâ€™s getting his oil checked!', "Gee, someone is upset that they are trying to give Fox a run for their money.  Don't worry Sierra, Fox still controls that niche.", "What are they being false about?  It's a fact that the military assets are being moved.", 'Bernie is the most consistent and most  trustworthy progressive. Feel the Bern!', 'Dennis Manson Only wimps, perverts, and lazy men vote Democrat.  Which ones are you?', 'Melania\'s body language is so stiff. I do not like her at all. What\'s with "be best"?', "USA Freedom Patriots made the Intern

In [23]:
dataset_claim_embeds.columns.to_list()

['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'commentText',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',


In [ ]:
YT_input_sort = YT_input.sort_values(by='parsed_datetime').reset_index(drop=True)

In [162]:
#Expand the polposts of all YT comments to include the thread context:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
YT_input_sort = YT_input.sort_values(by='parsed_datetime').reset_index(drop=False)
groupeddata = YT_input_sort[:20].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.index.to_series().shift(1)
    print(df.loc[:, ['dataset_index', 'commentText', 'preceding_index']])
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            chunked_result.append(
                pd.DataFrame(
                    data=[[row['dataset_index'], row['commentText']]],
                    columns=['index', 'post_expansion']
                )
            )
        if pd.notna(row['preceding_index']):
            #threadset = int(row['preceding_index'])
            print(f'"Thread":\n<{df["commentText"][:index].to_list()}>, "Target reply":<<{df["commentText"].iloc[index]}>>')
                                

100%|██████████| 16/16 [00:00<00:00, 368.99it/s]

   dataset_index  \
0            196   

                                                                                           commentText  \
0  No, the industry is worse! If you to took a biologylesson you would learn that the biodiversity ...   

   preceding_index  
0              NaN  
   dataset_index  \
0            190   

                                                                                           commentText  \
0  i saw the show. it was amazing. go see it, you wont regret it! and i love the little sound clips...   

   preceding_index  
0              NaN  
   dataset_index           commentText  preceding_index
0            398  she looks like coal.              NaN
   dataset_index  \
0            177   

                                                      commentText  \
0  the only solution for it...... hire a rapist..... just kidding   

   preceding_index  
0              NaN  
   dataset_index  \
0            185   

                                 

In [56]:
#Expand the YT comments to include the thread context:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
YT_input_sort = YT_input.sort_values(by='parsed_datetime').reset_index(drop=False)
groupeddata = YT_input_sort.groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            chunked_result.append(
                pd.DataFrame(
                    data=[[row['dataset_index'], row['commentText']]],
                    columns=['index', 'post_expansion']
                )
            )
        if pd.notna(row['preceding_index']):
            #threadset = int(row['preceding_index'])
            retry_count = 0
            max_retries = 10
           
            while retry_count < max_retries:
                try: 
                    response = requests.post(
                            url=api_endpoint,
                            headers=headers,
                            json={
                                'model': MODELgpt4o,
                                'messages': [
                                    {
                                        "role": "system",
                                        "content": SYSTEM_expansion
                                    },
                                    {
                                        "role": "user",
                                        "content": f'"Thread":\n<{df["commentText"][:index].to_list()}>, "Target reply":<<{df["commentText"].iloc[index]}>>',
                                    }
                                ],
                                'temperature': temperature_0,  
                                'seed': SEED,
                                
                            }
                        )  
                    if response.status_code == 200:
                        data_response = response.json()
                        chunked_result.append(
                        pd.DataFrame(
                            data=[[row['dataset_index'], data_response["choices"][0]["message"]["content"]]],                                
                            columns=['index', 'post_expansion']
                            )
                        )
                        break  # Exit the retry loop on success
                    elif response.status_code == 429:
                        retry_count += 1
                        retry_after = response.headers.get("Retry-After")
                        error_message = response.json().get("error", {}).get("message", "")
                        retry_after = 30  # Default to 30 seconds if not found

                        # Extract retry time from the error message
                        if "Try again in" in error_message:
                            match = re.search(r"Try again in (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                        elif "Please retry after" in error_message:
                            match = re.search(r"Please retry after (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                                
                        else:
                            retry_after = 30  # Default to 30 seconds if not found
                            print(f"Rate limit exceeded. Defaulting to retry in {retry_after} seconds.")

                        print(f"Rate limit exceeded. Retrying in {retry_after} seconds: {response.json()}. Retry count = {retry_count}") 
                        time.sleep(retry_after)

                    #    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                    #    print(response.text)
                    #    time.sleep(wait_time)
                    elif response.status_code == 500:
                        retry_count += 1
                        wait_time = 20
                        print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to connect to API. Status code: {response.status_code}")
                        print(response.text)
                        break
                except requests.exceptions.RequestException as e:   
                    print(f"Failed to connect to API: {e}")
                    retry_count += 1
                    wait_time = 60
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)                 

expanded_YT_posts = pd.concat(chunked_result, ignore_index=True)
print(expanded_YT_posts)

  0%|          | 4/1137 [00:12<58:41,  3.11s/it]  

Rate limit exceeded. Retrying in 24 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 24 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1


  1%|          | 12/1137 [01:02<1:43:27,  5.52s/it]

Rate limit exceeded. Retrying in 1 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 60 seconds: {'error': {'message': "No deployments available for selected model, Try again in 60 seconds. Passed model=nf-gpt-4o-2024-08-06. pre-call-checks=False, allowed_model_region=n/a, cooldown_list=[('717f00f7-34de-499b-96a8-0e12308b6b72', {'Exception Received': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API 

  1%|          | 14/1137 [02:20<4:02:37, 12.96s/it]

...

Rate limit exceeded. Retrying in 17 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 17 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1


 99%|█████████▊| 1121/1137 [1:16:54<00:31,  1.95s/it]

Rate limit exceeded. Retrying in 12 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 12 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 1 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nR

100%|██████████| 1137/1137 [1:18:42<00:00,  4.15s/it]

      index  \
0      2936   
1      3573   
2      2864   
3      3098   
4      3861   
...     ...   
3127   3787   
3128   1746   
3129   1552   
3130   3035   
3131   2362   

                                                                                           post_expansion  
0                                                                        sam at 3:24 LMAO run for ur life  
1                                                 Corey Hostetler that's what the bull represents.....duh  
2                                                                                      Lux you’re welcome  
3                                                                            Stop killing inocent people!  
4     The Syrian crisis is a horrible thing, but has anyone noticed the situation’s strange components...  
...                                                                                                   ...  
3127  @Scorp308 There are not really many good options in Syria

In [62]:
expanded_YT_posts = pd.concat(chunked_result, ignore_index=True)
expanded_YT_posts.set_index(expanded_YT_posts['index'], inplace=True)


In [63]:
#new run 2: now without token output limitation:
expanded_YT_posts

,index,post_expansion
index,,
2936,2936,sam at 3:24 LMAO run for ur life
3573,3573,Corey Hostetler that's what the bull represents.....duh
2864,2864,Lux you’re welcome
3098,3098,Stop killing inocent people!
3861,3861,"The Syrian crisis is a horrible thing, but has anyone noticed the situation’s strange components..."
...,...,...
3787,3787,"@Scorp308 There are not really many good options in Syria right now, but doing nothing or suppor..."
1746,1746,"Assad has three children, so it's probably them who were downloading those songs."
1552,1552,+ZenRules - you have made that mistake in Afghanistan and out came Bin Laden. Assad was responsi...


In [165]:
#new run, with better listing of preceding comments, i.e. not including the target comment:
expanded_YT_posts

,index,post_expansion
index,,
2936,2936,sam at 3:24 LMAO run for ur life
3573,3573,Corey Hostetler that's what the bull represents.....duh
2864,2864,Lux you’re welcome
3098,3098,Stop killing inocent people!
3861,3861,"The Syrian crisis is a horrible thing, but has"
...,...,...
3787,3787,@Scorp308 There are not really many good
1746,1746,"Assad has three children, so it's probably them"
1552,1552,+ZenRules - you have made that mistake in


In [196]:
expanded_YT_posts

,index,post_expansion
index,,
2936,2936,sam at 3:24 LMAO run for ur life
3573,3573,Corey Hostetler that's what the bull represents.....duh
2864,2864,Lux you’re welcome
3098,3098,Stop killing innocent people involved in the Syrian crisis!
3861,3861,"The Syrian crisis is a horrible thing, but has"
...,...,...
3787,3787,"@Scorp308, considering the current situation in"
1746,1746,"Assad has three children, so it's probably them"
1552,1552,+ZenRules - you have made that mistake in


In [68]:
#read legacy data:
YT_claim_embeds_leg = pd.read_parquet(f'{CFG.report_dir}/pubsphere_YT_posts.parquet')

In [69]:
YT_claim_embeds_leg.drop(columns=['post_expansion_gpt4o'], inplace=True)

In [ ]:
YT_input.merge(expanded_YT_posts, left_on='dataset_index', right_index=True, how='left').loc[:, ['dataset_index', 'commentText', 'post_expansion']]

In [67]:
YT_inputmerge = YT_input.merge(expanded_YT_posts, left_on='dataset_index', right_index=True, how='left')

In [72]:
YT_claim_embeds_leg.merge(YT_inputmerge.loc[:, ['dataset_index', 'Mark_ID', 'post_expansion']], left_on='Mark_ID', right_on='Mark_ID', how='left').columns

Index(['StartDate', 'RecordedDate', 'IPAddress', 'Finished', 'Coder', 'ID',
       'Mark_ID', 'Genre', 'topiccode', 'Platform',
       ...
       'most_similar_index_gpt4o', 'similarity_score_gpt4o',
       'commentText_most_sim_gpt4o', 'claims_ext_gpt4o_token10', 'post_index',
       'most_similar_ext_ID_gpt4o_token10',
       'similarity_score_ext_gpt4o_token10',
       'post_expenasion_most_sim_gpt4o_token10', 'dataset_index',
       'post_expansion'],
      dtype='object', length=123)

In [ ]:
YT_claim_embeds = YT_claim_embeds_leg.merge(YT_inputmerge.loc[:, ['dataset_index', 'Mark_ID', 'post_expansion']], left_on='Mark_ID', right_on='Mark_ID', how='left')
#YT_claim_embeds.drop(columns=['post_expansion_gpt4o'], inplace=True)
YT_claim_embeds.rename(columns={'post_expansion': 'post_expansion_gpt4o'}, inplace=True)
YT_claim_embeds.loc[:, ['commentText', 'post_expansion_gpt4o']].tail(5)

In [ ]:
YT_claim_embeds

In [24]:
YT_claim_embeds.rename(columns={'post_expansion_gpt4o_tokenlimit10': 'post_expansion_gpt4o_token10', 'most_similar_ext_ID_gpt4o':'most_similar_ext_ID_gpt4o_token10', 'similarity_score_ext_gpt4o':'similarity_score_ext_gpt4o_token10', 'post_expenasion_most_sim_gpt4o':'post_expenasion_most_sim_gpt4o_token10', 'claims_ext_gpt4o':'claims_ext_gpt4o_token10'}, inplace=True)

In [75]:
#save data:
YT_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere_YT_posts.parquet')

In [17]:
dataset_claim_embeds.rename(columns={'post_expansion_gpt4o': 'post_expansion_gpt4o_token10', 'most_similar_ext_ID_gpt4o':'most_similar_ext_ID_gpt4o_token10', 'similarity_score_ext_gpt4o':'similarity_score_ext_gpt4o_token10', 'post_expenasion_most_sim_gpt4o':'post_expenasion_most_sim_gpt4o_token10'}, inplace=True)

In [22]:
dataset_claim_embeds.rename(columns={'claims_ext_gpt4o':'claims_ext_gpt4o_token10'}, inplace=True)

In [80]:
dataset_claim_embeds = dataset_claim_embeds.merge(YT_inputmerge.loc[:, ['dataset_index', 'Mark_ID', 'post_expansion']], left_on='Mark_ID', right_on='Mark_ID', how='left')

In [81]:
dataset_claim_embeds.columns

Index(['StartDate', 'RecordedDate', 'IPAddress', 'Finished', 'Coder', 'ID',
       'Mark_ID', 'Genre', 'topiccode', 'Platform',
       ...
       'either_political', 'post_ada_embedding', 'dataset_index_x',
       'post_expansion_gpt4o_token10', 'most_similar_index_gpt4o',
       'similarity_score_gpt4o', 'commentText_most_sim_gpt4o',
       'claims_ext_gpt4o_token10', 'dataset_index_y', 'post_expansion'],
      dtype='object', length=118)

In [83]:
dataset_claim_embeds.drop(columns=['dataset_index_y'], inplace=True)

In [82]:
#dataset_claim_embeds.drop(columns=['post_expansion_gpt4o'], inplace=True)
dataset_claim_embeds.rename(columns={'post_expansion': 'post_expansion_gpt4o'}, inplace=True)


In [201]:
dataset_claim_embeds = dataset_claim_embeds.join(expanded_YT_posts['post_expansion'], rsuffix='_gpt4o')
dataset_claim_embeds.rename(columns={'post_expansion': 'post_expansion_gpt4o'}, inplace=True)

In [85]:
#save data:
dataset_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')

In [34]:
#mine claims from the original posts as a benchmark

chunked_result: typing.List[pd.DataFrame] = []
for index, row in tqdm.tqdm(YT_claim_embeds.iterrows()):
    retry_count = 0
    max_retries = 10
    
    while retry_count < max_retries:
        try: 
            response = requests.post(
                    url=api_endpoint,
                    headers=headers,
                    json={
                        'model': MODELgpt4o,
                        'messages': [
                            {
                                "role": "system",
                                "content": SYSTEM_claim                         
                            },
                            {
                                "role": "user",
                                "content": f'The following social media post is a reply to a news- or infotainment video. '
                                         + f'Check whether your answer strictly adheres to the specified format. \n"Post":\n<{row["commentText"]}>',
                            }
                        ],
                        'temperature': temperature_0,  
                        'seed': SEED
                    }
                )
            
            if response.status_code == 200:
                data_response = response.json()
                chunked_result.append(
                pd.DataFrame(
                    data=[[row['dataset_index'], data_response["choices"][0]["message"]["content"]]],                                
                    columns=['index', 'claims']
                    )
                )
                break  # Exit the retry loop on success  
            elif response.status_code == 429:
                retry_count += 1
                retry_after = response.headers.get("Retry-After")
                error_message = response.json().get("error", {}).get("message", "")
                retry_after = 30  # Default to 30 seconds if not found
            
                # Extract retry time from the error message
                if "Try again in" in error_message:
                    match = re.search(r"Try again in (\d+) second", error_message)
                    if match:
                        try:
                            retry_after = int(match.group(1))
                        except (IndexError, ValueError) as e:
                            print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                            pass
                elif "Please retry after" in error_message:
                    match = re.search(r"Please retry after (\d+) second", error_message)
                    if match:
                        try:
                            retry_after = int(match.group(1))
                        except (IndexError, ValueError) as e:
                            print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                            pass
                

                else:
                    retry_after = 30  # Default to 30 seconds if not found
                    print(f"Rate limit exceeded. Defaulting to retry in {retry_after} seconds.")
                            
        
                print(f"Rate limit exceeded. Retrying in {retry_after} seconds: {response.json()}. Retry count = {retry_count}") 
                time.sleep(retry_after)
                
            #    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
            #    print(response.text)
            #    time.sleep(wait_time)
            elif response.status_code == 500:
                retry_count += 1
                wait_time = 20
                print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                print(response.text)
                time.sleep(wait_time)
            else:
                print(f"Failed to connect to API. Status code: {response.status_code}")
                print(response.text)
                break
        except requests.exceptions.RequestException as e:   
            print(f"Failed to connect to API: {e}")
            retry_count += 1
            wait_time = 60
            print(f"Retrying in {wait_time} seconds...")
            time.sleep(wait_time)   

        if retry_count >= max_retries:
            print(f"Max retries reached for index {index}. Skipping to next item.")
            break  # Exit the loop if max retries are reached              

YT_claims = pd.concat(chunked_result, ignore_index=True)
print(YT_claims.head())  

57it [00:58,  1.69it/s]

Rate limit exceeded. Retrying in 2 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1


64it [01:03,  2.20it/s]

Rate limit exceeded. Retrying in 4 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 4 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 60 seconds: {'error': {'message': "No deployments available for selected model, Try again in 60 seconds. Passed model=nf-gpt-4o-2024-08-06. pre-call-checks=False, allowed_model_region=n/a, cooldown_list=[('717f00f7-34de-499b-96a8-0e12308b6b72', {'Exception Received': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API

121it [02:51,  2.41it/s]

...

Rate limit exceeded. Retrying in 5 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 5 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 60 seconds: {'error': {'message': "No deployments available for selected model, Try again in 60 seconds. Passed model=nf-gpt-4o-2024-08-06. pre-call-checks=False, allowed_model_region=n/a, cooldown_list=[('717f00f7-34de-499b-96a8-0e12308b6b72', {'Exception Received': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API

3129it [1:39:13,  1.89it/s]

Rate limit exceeded. Retrying in 7 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1


3132it [1:39:26,  1.91s/it]

   index  \
0      0   
1      1   
2      3   
3      4   
4      5   

                                                                   claims  
0                                                                      []  
1                                            ["Minerals have rights too"]  
2  ["The death of free and civil dialogue has been effectively depicted"]  
3                                            ["No-one else will hug him"]  
4                                                                      []  


In [36]:
chunked_result

[   index claims
 0      0     [],
    index                        claims
 0      1  ["Minerals have rights too"],
    index  \
 0      3   
 
                                                                    claims  
 0  ["The death of free and civil dialogue has been effectively depicted"]  ,
    index                        claims
 0      4  ["No-one else will hug him"],
    index claims
 0      5     [],
    index                                               claims
 0      6  ["Trump is a traitor", "Trump should be locked up"],
    index claims
 0      7     [],
    index  \
 0      8   
 
                                                                                                 claims  
 0  ["America is a country of idiots run by the NRA", "America should get rid of the 2nd amendment",...  ,
    index                                            claims
 0      9  ["They weren't going to vote democratic anyway"],
    index                                         claims
 0  

In [35]:
YT_claims = pd.concat(chunked_result, ignore_index=True)
print(YT_claims.head())  

   index  \
0      0   
1      1   
2      3   
3      4   
4      5   

                                                                   claims  
0                                                                      []  
1                                            ["Minerals have rights too"]  
2  ["The death of free and civil dialogue has been effectively depicted"]  
3                                            ["No-one else will hug him"]  
4                                                                      []  


In [210]:
YT_claims.set_index(YT_claims['index'], inplace=True)
YT_claim_embeds = YT_claim_embeds.join(YT_claims['claims'])
YT_claim_embeds.rename(columns={'claims': 'claims_gpt4o'}, inplace=True)

In [213]:
#save data:
YT_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere_YT_posts.parquet')

In [7]:
#load data: 
YT_claim_embeds = pd.read_parquet(f'{CFG.report_dir}/pubsphere_YT_posts.parquet')

In [9]:
YT_claim_embeds.columns

Index(['StartDate', 'RecordedDate', 'IPAddress', 'Finished', 'Coder', 'ID',
       'Mark_ID', 'Genre', 'topiccode', 'Platform',
       ...
       'political_post_gpt4o', 'political_post_gpt4o_dum', 'either_political',
       'Time_comment_dt', 'post_ada_embedding', 'claims_gpt4o',
       'post_expansion_gpt4o', 'most_similar_index_gpt4o',
       'similarity_score_gpt4o', 'commentText_most_sim_gpt4o'],
      dtype='object', length=116)

In [86]:
#mine claims from the expanded posts:

chunked_result2: typing.List[pd.DataFrame] = []
for index, row in tqdm.tqdm(YT_claim_embeds.iterrows()):
    retry_count = 0
    max_retries = 10
    
    while retry_count < max_retries:
        try: 
            response = requests.post(
                    url=api_endpoint,
                    headers=headers,
                    json={
                        'model': MODELgpt4o,
                        'messages': [
                            {
                                "role": "system",
                                "content": SYSTEM_claim                         
                            },
                            {
                                "role": "user",
                                "content": f'The following social media post is a reply to a news- or infotainment video. '
                                         + f'Check whether your answer strictly adheres to the specified format. \n"Post":\n<{row["post_expansion_gpt4o"]}>',
                            }
                        ],
                        'temperature': temperature_0,  
                        'seed': SEED
                    }
                )
            
            if response.status_code == 200:
                data_response = response.json()
                chunked_result2.append(
                pd.DataFrame(
                    data=[[row['Mark_ID'], data_response["choices"][0]["message"]["content"]]],                                
                    columns=['index', 'claims']
                    )
                )
                break  # Exit the retry loop on success  
            elif response.status_code == 429:
                retry_count += 1
                retry_after = response.headers.get("Retry-After")
                error_message = response.json().get("error", {}).get("message", "")
                retry_after = 30  # Default to 30 seconds if not found
            
                # Extract retry time from the error message
                if "Try again in" in error_message:
                    match = re.search(r"Try again in (\d+) second", error_message)
                    if match:
                        try:
                            retry_after = int(match.group(1))
                        except (IndexError, ValueError) as e:
                            print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                            pass
                elif "Please retry after" in error_message:
                    match = re.search(r"Please retry after (\d+) second", error_message)
                    if match:
                        try:
                            retry_after = int(match.group(1))
                        except (IndexError, ValueError) as e:
                            print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                            pass
                

                else:
                    retry_after = 30  # Default to 30 seconds if not found
                    print(f"Rate limit exceeded. Defaulting to retry in {retry_after} seconds.")
                            
        
                print(f"Rate limit exceeded. Retrying in {retry_after} seconds: {response.json()}. Retry count = {retry_count}") 
                time.sleep(retry_after)
                
            #    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
            #    print(response.text)
            #    time.sleep(wait_time)
            elif response.status_code == 500:
                retry_count += 1
                wait_time = 20
                print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                print(response.text)
                time.sleep(wait_time)
            else:
                print(f"Failed to connect to API. Status code: {response.status_code}")
                print(response.text)
                break
        except requests.exceptions.RequestException as e:   
            print(f"Failed to connect to API: {e}")
            retry_count += 1
            wait_time = 60
            print(f"Retrying in {wait_time} seconds...")
            time.sleep(wait_time)   

        if retry_count >= max_retries:
            print(f"Max retries reached for index {index}. Skipping to next item.")
            break  # Exit the loop if max retries are reached              

YT_exp_claims = pd.concat(chunked_result2, ignore_index=True)
print(YT_exp_claims.head())  

56it [00:28,  3.00it/s]

Rate limit exceeded. Retrying in 33 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 33 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1


80it [01:10,  3.13it/s]

Rate limit exceeded. Retrying in 2 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 60 seconds: {'error': {'message': "No deployments available for selected model, Try again in 60 seconds. Passed model=nf-gpt-4o-2024-08-06. pre-call-checks=False, allowed_model_region=n/a, cooldown_list=[('717f00f7-34de-499b-96a8-0e12308b6b72', {'Exception Received': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API

137it [03:00,  2.15s/it]

...

Rate limit exceeded. Retrying in 35 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 35 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1


3125it [1:30:02,  1.92it/s]

Rate limit exceeded. Retrying in 29 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 29 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1


3132it [1:30:36,  1.74s/it]

   index  \
0    119   
1    282   
2    769   
3   1206   
4   1214   

                                                                   claims  
0                                                                      []  
1                                            ["Minerals have rights too"]  
2  ["The death of free and civil dialogue has been effectively depicted"]  
3                                            ["No-one else will hug him"]  
4                                                                      []  


In [87]:
#new run:
YT_exp_claims

,index,claims
0,119,[]
1,282,"[""Minerals have rights too""]"
2,769,"[""The death of free and civil dialogue has been effectively depicted""]"
3,1206,"[""No-one else will hug him""]"
4,1214,[]
...,...,...
3127,2002575,[]
3128,3000508,[]
3129,20000102,"[""They knew all about the cameras"", ""The bald guy probably has a head of hair"", ""The red haired ..."
3130,20000418,[]


In [18]:
YT_exp_claims

,index,claims
index,,
119,119,[]
282,282,[]
769,769,"[""The death of free and civil dialogue has been effectively depicted""]"
1206,1206,"[""No-one else will hug him.""]"
1214,1214,[]
...,...,...
2002575,2002575,[]
3000508,3000508,[]
20000102,20000102,"[""They knew all about the cameras""]"


In [89]:
YT_exp_claims.set_index(YT_exp_claims['index'], inplace=True)
YT_claim_embeds = YT_claim_embeds.join(YT_exp_claims['claims'], on='Mark_ID', rsuffix='_ext_gpt4o')
YT_claim_embeds.rename(columns={'claims': 'claims_ext_gpt4o'}, inplace=True)

In [ ]:
YT_claim_embeds.loc[:, ['commentText', 'post_expansion_gpt4o', 'claims_ext_gpt4o']].head(5)


In [91]:
#save data:
YT_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere_YT_posts.parquet')

In [95]:
dataset_claim_embeds.drop(columns=['claims_ext_gpt4o'], inplace=True)

In [101]:
dataset_claim_embeds = dataset_claim_embeds.merge(YT_claim_embeds.loc[:,['claims_ext_gpt4o', 'Mark_ID']], how='left', on='Mark_ID')

In [103]:
#save data:
dataset_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')

In [ ]:
SYSTEM_new: str = \
    """
# Instruction

Identify whether a social media posts adds new information (2), expands on information (1) or does not add new information (0) on the topic of a debate. New information is present (2) if the post introduces a new argument or perspective within the thread topic. 
A post expands on an existing argument or perspective (1) if it provides additional context or examples to an existing argument or perspective, or if it provides a sub-argument (parent-child relation). 
If the post does not relate to the thread topic or provides no new information, assign a value of 0. Follow these steps:


Follow these steps:

1. You will receive a target post in double chevrons <<...>> along with the thread topic in single chevrons <...>, and preceding posts enclosed in triple chevrons `<<<...>>>`.
2. Verify if the target post is on the thread topic.
3. Identify if new information is presented in the target post with respect to the preceding posts. 
4. If in doubt, assign a value of 0.

Respond with only the predicted class (0 or 1 or 2) of the request. Do not include any additional text or explanations.
Class:
	"""

In [ ]:
SYSTEM_claim_new: str = \
    """
# Instruction

Identify whether a claim adds new information (2), expands on information (1) or does not add new information (0) with respect to preceding claims. New information is present (2) if the claim introduces a new argument or perspective. 
A claim expands on an existing argument or perspective (1) if it strongly related to it, or it provides additional context or examples to an argument or perspective in a preceding claim, or if it provides a sub-argument (parent-child relation). 
If the claim is very similar to a preceding claim or does not add any perspective or argument, assign a value of 0. Follow these steps:


Follow these steps:

1. You will receive a target claim in double chevrons <<...>> along with the preceding claims in single chevrons <...>.
2. Identify if new information is presented in the target claim with respect to the preceding posts. 
4. If in doubt, assign a value of 0.

Respond with only the predicted class (0 or 1 or 2) of the request. Do not include any additional text or explanations.
Class:
	"""

In [94]:
#get ADA embeddings for posts:
YT_claim_embeds.loc[:, 'post_ada_embedding'] = YT_claim_embeds['commentText'].progress_apply(get_ada_embedding)


100%|██████████| 3132/3132 [14:26<00:00,  3.62it/s] 


In [96]:
YT_claim_embeds.loc[:, 'post_ada_embedding']


0       [0.0012491943780332804, -0.01312993373721838, 0.004941582679748535, -0.026987144723534584, -0.01...
1       [-0.012995055876672268, -0.016615739092230797, 0.018477026373147964, -0.013490946963429451, -0.0...
3       [-0.0008296146406792104, 0.0013742600567638874, 0.010784990154206753, -0.030297795310616493, -0....
4       [-0.026431242004036903, -0.003406326286494732, 0.005844608414918184, -0.01946660876274109, -0.00...
5       [-0.03259480744600296, 0.0038548349402844906, -0.006202991586178541, 0.008793946355581284, -0.03...
                                                       ...                                                 
3854    [-0.0018265506951138377, -0.02517927996814251, -0.0021120987366884947, -0.015046310611069202, -0...
3855    [-0.005544822663068771, -0.017791016027331352, -0.00620570732280612, -0.019258178770542145, -0.0...
3859    [0.002804453717544675, -0.009028810076415539, 0.024697821587324142, -0.02455144375562668, 0.0020...
3860    [-0.0244190171360969

In [97]:
#save data:
YT_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere_YT_posts.parquet')

In [110]:
dataset_claim_embeds = dataset_claim_embeds.join(YT_claim_embeds['post_ada_embedding'])

In [112]:
#get ada embeddings for post with missing ada embedding values:
missing_indices = dataset_claim_embeds['post_ada_embedding'].isna()
dataset_claim_embeds.loc[missing_indices, 'post_ada_embedding'] = dataset_claim_embeds.loc[missing_indices, 'commentText'].progress_apply(get_ada_embedding)

100%|██████████| 730/730 [03:20<00:00,  3.64it/s]


In [114]:
dataset_claim_embeds.head()


,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,cosine_similarity_post_claim_MXBAI,cosine_low_high_MXBAI,tfidf_embed_post_svd,Llama31_political_post_8b,Llama31_political_fill_8b,Llama31_political_fill_8b_score,political_post_gpt4o,political_post_gpt4o_dum,either_political,post_ada_embedding
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,0.960095,1.000000,[1.0],non-political,non-political,0,non-political,0,0,"[0.0012491943780332804, -0.01312993373721838, 0.004941582679748535, -0.026987144723534584, -0.01..."
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,0.957984,0.991973,[0.9999999959466253],political,political,1,non-political,0,1,"[-0.012995055876672268, -0.016615739092230797, 0.018477026373147964, -0.013490946963429451, -0.0..."
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,0.967763,0.996136,[1.0000000354648362],political,political,1,political,1,1,"[-0.0028026909567415714, 0.007603764999657869, 0.022936813533306122, -0.055016644299030304, -0.0..."
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,0.952258,0.992674,[0.9999999904984119],political,political,1,non-political,0,1,"[-0.0008296146406792104, 0.0013742600567638874, 0.010784990154206753, -0.030297795310616493, -0...."
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,0.977981,0.992789,[1.0],non-political,non-political,0,non-political,0,0,"[-0.026431242004036903, -0.003406326286494732, 0.005844608414918184, -0.01946660876274109, -0.00..."


In [115]:
#save data:
dataset_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')

In [63]:
#get embeddings for the claims:
#first load the claims:
YT_claim_embeds = pd.read_parquet(f'{CFG.report_dir}/pubsphere_YT_posts.parquet')


In [6]:
YT_claim_embeds.head()

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,tfidf_embed_post_svd,Llama31_political_post_8b,Llama31_political_fill_8b,Llama31_political_fill_8b_score,political_post_gpt4o,political_post_gpt4o_dum,either_political,Time_comment_dt,post_expansion_gpt4o,post_ada_embedding
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,[1.0],non-political,non-political,0,non-political,0,0,2017-11-16 09:14:09+00:00,Is 1:37 the new 60 minutes,"[0.0012491943780332804, -0.01312993373721838, 0.004941582679748535, -0.026987144723534584, -0.01..."
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,[0.9999999959466253],political,political,1,non-political,0,1,2019-08-12 20:22:06+00:00,the only solution for it...... hire a rapist..... just kidding,"[-0.012995055876672268, -0.016615739092230797, 0.018477026373147964, -0.013490946963429451, -0.0..."
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,[0.9999999904984119],political,political,1,non-political,0,1,2018-10-03 05:44:24+00:00,"@TheJohny815 true. dinosaurs microscopic organisms, neanderthals and other pre-historic humans...","[-0.0008296146406792104, 0.0013742600567638874, 0.010784990154206753, -0.030297795310616493, -0...."
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,[1.0],non-political,non-political,0,non-political,0,0,2018-08-28 05:38:51+00:00,"No, the industry is worse! If you to took a biologylesson you would learn that the biodiversity ...","[-0.026431242004036903, -0.003406326286494732, 0.005844608414918184, -0.01946660876274109, -0.00..."
5,11/30/2021 17:24:18,11/30/2021 17:27:16,213.127.109.191,1,6,Ugw2eTvkZLfH9MDVg1R4AaABAg,1214,1,2,1,...,[1.000000006283446],political,political,1,political,1,1,2019-08-20 00:31:48+00:00,go mitt Romney 2012,"[-0.03259480744600296, 0.0038548349402844906, -0.006202991586178541, 0.008793946355581284, -0.03..."


In [11]:
YT_input

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,likeCount_video,date_difference,commentCount_video,replyCount_comment,topic,subscribers,HATELIST_FOCUSED_DUMMY,Time_comment_year,Time_video_year,dataset_index
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2017,2017.0,0
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,3817.0,743.0,1748.0,NaN,economy,3630000.0,0,2019,2019.0,1
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0,3
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0,4
5,11/30/2021 17:24:18,11/30/2021 17:27:16,213.127.109.191,1,6,Ugw2eTvkZLfH9MDVg1R4AaABAg,1214,1,2,1,...,9472.0,743.0,2912.0,0.0,economy,3630000.0,1,2019,2018.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3854,12/31/2021 15:07:40,12/31/2021 15:08:38,193.154.174.158,1,6,UgwiL6YZam5Xg4FWUHh4AaABAg,2002575,0,3,1,...,776.0,2574.0,1004.0,0.0,east,105000.0,0,2019,2019.0,3854
3855,10/19/2021 22:23:07,10/19/2021 22:24:30,213.127.109.191,1,6,UgxGubygP72yCR0-la14AaABAg,3000508,0,3,1,...,12475.0,3803.0,4785.0,0.0,east,6740000.0,0,2019,2010.0,3855
3859,10/6/2021 16:08:39,10/6/2021 16:10:42,213.127.113.113,1,6,UghFY3QJ6nmT_ngCoAEC.7-H0Z7--wxd8goqpaPs-bl,20000102,0,3,1,...,12475.0,3803.0,4785.0,NaN,east,6740000.0,0,2018,2010.0,3859
3860,10/15/2021 18:30:04,10/15/2021 18:35:40,213.127.109.191,1,6,UgyWabsmmnq3zam4DgZ4AaABAg,20000418,2,3,1,...,31761.0,1531.0,2206.0,0.0,east,6800000.0,0,2018,2015.0,3860


In [90]:
test = YT_input.sort_values(by='parsed_datetime', ascending=True).reset_index(drop=False).iloc[:50]
test['preceding_index'] = test.index.to_series().shift(1)

In [123]:

for group, df in tqdm.tqdm(test[11:13].groupby(GROUPER)):
    df.reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            print(threadset)
            preceding_comments = df.loc[:, ['dataset_index', 'commentText']].iloc[:index]
            print(f'"Target comment":<<{row["commentText"]}>>, index:{index}, preceding index:{threadset}, "preceding comments":\n<{preceding_comments.to_json(orient="records")}>')

100%|██████████| 1/1 [00:00<00:00, 490.68it/s]

0
"Target comment":<<If your criteria begins and ends with "hot".>>, index:1, preceding index:0, "preceding comments":
<[{"dataset_index":331,"commentText":"She looks like a pron star I don't \u00c2\u00a0but to me she has like a porn star vibe about her like she's someone you'll see on bang bus."}]>


In [134]:
#get post simscore per thread:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
YT_input_sort = YT_input.sort_values(by='parsed_datetime', ascending=True).reset_index(drop=False)
groupeddata = YT_input_sort[11:13].groupby(GROUPER)
#groupeddata = dataset_claim_embeds.commentText[1:10].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.index.to_series().shift(1)
    print('df:')
    print(df.loc[:,['dataset_index', 'commentText', 'preceding_index','videoTitle', 'parsed_datetime']])
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            print("preceding comment:" + df.commentText[threadset])
            print('preceding df:')
            print(df.loc[:,['dataset_index', 'commentText', 'preceding_index','videoTitle', 'parsed_datetime']].iloc[:int(row['preceding_index'])])
            print(f'"Target comment":<<{row["commentText"]}>>, index:{index}, dataset_index:{row['dataset_index']} preceding index:{row['preceding_index']}, "preceding comments":\n<{df.loc[:,['dataset_index', 'commentText']][:index].to_json(orient='records')}>')

100%|██████████| 1/1 [00:00<00:00, 154.98it/s]

df:
   dataset_index  \
0            331   
1            330   

                                                                                           commentText  \
0  She looks like a pron star I don't Â but to me she has like a porn star vibe about her like she'...   
1                                                         If your criteria begins and ends with "hot".   

   preceding_index  \
0              NaN   
1              0.0   

                                                          videoTitle  \
0  Real Time with Bill Maher: Haters Gonna Hate with S.E. Cupp (HBO)   
1  Real Time with Bill Maher: Haters Gonna Hate with S.E. Cupp (HBO)   

            parsed_datetime  
0 2014-02-14 23:33:54+00:00  
1 2014-02-15 03:22:49+00:00  
preceding comment:She looks like a pron star I don't Â but to me she has like a porn star vibe about her like she's someone you'll see on bang bus.
preceding df:
Empty DataFrame
Columns: [dataset_index, commentText, preceding_index, videoTitl

In [126]:
#get simscore for posts using Llama3.1:8b:

chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
YT_input_sort = YT_input.sort_values(by='parsed_datetime', ascending=True).reset_index(drop=False)
groupeddata = YT_input_sort[10:13].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            try: 
                chunked_result.append(pd.DataFrame(
                    data=[[requests.post(
                                'https://inf.cl.uni-trier.de/',
                                json={
                                    'model': MODELsmall,
                                    'system': SYSTEM_post_simscore,
                                    'prompt': f'"Target comment":<<{row["commentText"]}>>, "preceding comments":\n<{df.loc[:,['dataset_index', 'commentText']][:index].to_json(orient='records')}>',
                                    'options': options_zero
                                    }).json()['response']]],
                    columns=['similariy']
                ))
                
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")

similarity = pd.concat(chunked_result, ignore_index=True)
print(similarity)

100%|██████████| 2/2 [00:04<00:00,  2.29s/it]

                                                                                             similariy
0  [\n    {\n        "most_similar_comment_index": "1",\n        "similarity_score": "1"\n    }\n]\...


In [130]:
similarity['similariy'][0]

'[\n    {\n        "most_similar_comment_index": "1",\n        "similarity_score": "1"\n    }\n]\nThe target comment and the preceding comment are identical, so I\'ve assigned a similarity score of 10.'

In [183]:
#get simscore for posts using GPT4o:

chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
YT_input_sort = YT_input.sort_values(by='parsed_datetime', ascending=True).reset_index(drop=False)
groupeddata = YT_input_sort.groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            retry_count = 0
            max_retries = 10
    
            while retry_count < max_retries:
                try:  
                    response = requests.post(
                            url=api_endpoint,
                            headers=headers,
                            json={
                                'model': MODELgpt4o,
                                'messages': [
                                    {
                                        "role": "system",
                                        "content": SYSTEM_post_simscore                        
                                    },
                                    {
                                        "role": "user",
                                        "content": f'"Target comment":<<{row["commentText"]}>>, "preceding comments":\n<{df.loc[:,['dataset_index', 'commentText']][:index].to_json(orient='records')}>'
                                    
                                    }
                                ],
                                'temperature': temperature_0,  
                                'seed': SEED
                            }
                        )
                    
                    if response.status_code == 200:
                        data_response = response.json()
                        # Check if 'response' key exists in the JSON response
                        if 'choices' in data_response:
                            content = data_response["choices"][0]["message"]["content"]
                            # Remove code block markers if present
                            content = re.sub(r'```json|```', '', content).strip()
                            try:
                                content = json.loads(content)   
                                # Check if content_json is a list or a dictionary
                                if isinstance(content, dict):
                                    chunked_result.append({
                                        "post_index": row['dataset_index'],
                                        "most_similar_post_index": content.get('most_similar_comment_index', ''),
                                        'similarity_score': content.get('similarity_score', '')  
                                    })
                                elif isinstance(content, list):
                                    for item in content:
                                        chunked_result.append({
                                            "post_index": row['dataset_index'],
                                            "most_similar_post_index": item.get('most_similar_comment_index', ''),
                                            'similarity_score': item.get('similarity_score', '')  
                                        })
                                else:
                                    print(f"Unexpected content type: {type(content)}")

                            except json.JSONDecodeError as e:
                                print(f"Error decoding JSON: {e}")
                                print(data_response)
                        else:
                            print(f"No 'choices' key in response or 'choices' is empty: {content}")                        
                        break  # Exit the retry loop on success  
                    elif response.status_code == 429:
                        retry_count += 1
                        retry_after = response.headers.get("Retry-After")
                        error_message = response.json().get("error", {}).get("message", "")
                        retry_after = 30  # Default to 30 seconds if not found
                    
                        # Extract retry time from the error message
                        if "Try again in" in error_message:
                            match = re.search(r"Try again in (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                        elif "Please retry after" in error_message:
                            match = re.search(r"Please retry after (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                                
                                
                        else:
                            retry_after = 30  # Default to 30 seconds if not found
                            print(f"Rate limit exceeded. Defaulting to retry in {retry_after} seconds.")
                                    
                
                        print(f"Rate limit exceeded. Retrying in {retry_after} seconds: {response.json()}. Retry count = {retry_count}") 
                        time.sleep(retry_after)
                        
                    #    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                    #    print(response.text)
                    #    time.sleep(wait_time)
                    elif response.status_code == 500:
                        retry_count += 1
                        wait_time = 20
                        print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to connect to API. Status code: {response.status_code}")
                        print(response.text)
                        break
                except requests.exceptions.RequestException as e:   
                    print(f"Failed to connect to API: {e}")
                    retry_count += 1
                    wait_time = 60
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)   
        
                if retry_count >= max_retries:
                    print(f"Max retries reached for index {index}. Skipping to next item.")
                    break  # Exit the loop if max retries are reached              

# Convert the list of dictionaries to a DataFrame
similarity = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(similarity.head(4))

 

  0%|          | 4/1137 [00:19<1:28:45,  4.70s/it]

Rate limit exceeded. Retrying in 25 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 25 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 2 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\n

  1%|          | 14/1137 [02:31<3:35:39, 11.52s/it]

Rate limit exceeded. Retrying in 30 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 30 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1


  3%|▎         | 33/1137 [03:30<55:51,  3.04s/it]  

...

Rate limit exceeded. Retrying in 22 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 22 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 1 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nR

100%|█████████▉| 1132/1137 [1:29:07<00:23,  4.67s/it]

Rate limit exceeded. Retrying in 18 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 18 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1


100%|██████████| 1137/1137 [1:29:51<00:00,  4.74s/it]

   post_index most_similar_post_index similarity_score
0        3098                    2864                1
1        3861                    3098                2
2        2334                    3098                5
3        1841                    3861                5


In [184]:
similarity = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(similarity.head(4))

   post_index most_similar_post_index similarity_score
0        3098                    2864                1
1        3861                    3098                2
2        2334                    3098                5
3        1841                    3861                5


In [194]:
#add target comment and most similar comment to similarity:
similarity.set_index(similarity['post_index'], inplace=True)
similarity = similarity.join(YT_input_sort.set_index('dataset_index')['commentText'], rsuffix='_target')
similarity.loc[:, 'most_similar_post_index'] = similarity.loc[:, 'most_similar_post_index'].astype(int)
similarity = similarity.merge(YT_input_sort.loc[:, ['dataset_index', 'commentText']], left_on = 'most_similar_post_index', right_on = 'dataset_index' , suffixes=('', '_source'))
similarity.drop(columns=['dataset_index'], inplace=True)
similarity.rename(columns={'most_similar_post_index': 'most_similar_index_gpt4o', 'similarity_score': 'similarity_score_gpt4o', 'commentText_source':'commentText_most_sim_gpt4o' }, inplace=True)
YT_claim_embeds = YT_claim_embeds.join(similarity.loc[:, ['most_similar_index_gpt4o', 'similarity_score_gpt4o', 'commentText_most_sim_gpt4o']])

In [201]:
#save data:
YT_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere_YT_posts.parquet')

In [203]:
dataset_claim_embeds = dataset_claim_embeds.join(similarity.loc[:, ['most_similar_index_gpt4o', 'similarity_score_gpt4o', 'commentText_most_sim_gpt4o']])

In [ ]:
dataset_claim_embeds.loc[:, ['commentText', 'videoTitle','most_similar_index_gpt4o', 'similarity_score_gpt4o', 'commentText_most_sim_gpt4o']]

In [204]:
#save data:
dataset_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')

In [106]:
#get simscore for post EXPANSION using GPT4o:

chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
YT_claim_embeds_sort = YT_claim_embeds.sort_values(by='Time_comment_dt', ascending=True).reset_index(drop=False)
groupeddata = YT_claim_embeds_sort.groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            retry_count = 0
            max_retries = 10
    
            while retry_count < max_retries:
                try:  
                    response = requests.post(
                            url=api_endpoint,
                            headers=headers,
                            json={
                                'model': MODELgpt4o,
                                'messages': [
                                    {
                                        "role": "system",
                                        "content": SYSTEM_post_simscore                        
                                    },
                                    {
                                        "role": "user",
                                        "content": f'"Target comment":<<{row["post_expansion_gpt4o"]}>>, "preceding comments":\n<{df.loc[:,['Mark_ID', 'post_expansion_gpt4o']][:index].to_json(orient='records')}>'
                                    
                                    }
                                ],
                                'temperature': temperature_0,  
                                'seed': SEED
                            }
                        )
                    
                    if response.status_code == 200:
                        data_response = response.json()
                        # Check if 'response' key exists in the JSON response
                        if 'choices' in data_response:
                            content = data_response["choices"][0]["message"]["content"]
                            # Remove code block markers if present
                            content = re.sub(r'```json|```', '', content).strip()
                            try:
                                content = json.loads(content)   
                                # Check if content_json is a list or a dictionary
                                if isinstance(content, dict):
                                    chunked_result.append({
                                        "post_index": row['Mark_ID'],
                                        "most_similar_post_index": content.get('most_similar_comment_index', ''),
                                        'similarity_score': content.get('similarity_score', '')  
                                    })
                                elif isinstance(content, list):
                                    for item in content:
                                        chunked_result.append({
                                            "post_index": row['Mark_ID'],
                                            "most_similar_post_index": item.get('most_similar_comment_index', ''),
                                            'similarity_score': item.get('similarity_score', '')  
                                        })
                                else:
                                    print(f"Unexpected content type: {type(content)}")

                            except json.JSONDecodeError as e:
                                print(f"Error decoding JSON: {e}")
                                print(data_response)
                        else:
                            print(f"No 'choices' key in response or 'choices' is empty: {content}")                        
                        break  # Exit the retry loop on success  
                    elif response.status_code == 429:
                        retry_count += 1
                        retry_after = response.headers.get("Retry-After")
                        error_message = response.json().get("error", {}).get("message", "")
                        retry_after = 30  # Default to 30 seconds if not found
                    
                        # Extract retry time from the error message
                        if "Try again in" in error_message:
                            match = re.search(r"Try again in (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                        elif "Please retry after" in error_message:
                            match = re.search(r"Please retry after (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                                
                                
                        else:
                            retry_after = 30  # Default to 30 seconds if not found
                            print(f"Rate limit exceeded. Defaulting to retry in {retry_after} seconds.")
                                    
                
                        print(f"Rate limit exceeded. Retrying in {retry_after} seconds: {response.json()}. Retry count = {retry_count}") 
                        time.sleep(retry_after)
                        
                    #    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                    #    print(response.text)
                    #    time.sleep(wait_time)
                    elif response.status_code == 500:
                        retry_count += 1
                        wait_time = 20
                        print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to connect to API. Status code: {response.status_code}")
                        print(response.text)
                        break
                except requests.exceptions.RequestException as e:   
                    print(f"Failed to connect to API: {e}")
                    retry_count += 1
                    wait_time = 60
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)   
        
                if retry_count >= max_retries:
                    print(f"Max retries reached for index {index}. Skipping to next item.")
                    break  # Exit the loop if max retries are reached              

# Convert the list of dictionaries to a DataFrame
similarity_ext = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(similarity_ext.head(4))


  0%|          | 0/1137 [00:00<?, ?it/s]

  0%|          | 4/1137 [00:17<1:16:56,  4.07s/it]

Rate limit exceeded. Retrying in 37 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 37 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 1 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nR

  1%|          | 12/1137 [01:09<1:52:49,  6.02s/it]

Rate limit exceeded. Retrying in 2 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 60 seconds: {'error': {'message': "No deployments available for selected model, Try again in 60 seconds. Passed model=nf-gpt-4o-2024-08-06. pre-call-checks=False, allowed_model_region=n/a, cooldown_list=[('717f00f7-34de-499b-96a8-0e12308b6b72', {'Exception Received': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API

...

Rate limit exceeded. Retrying in 37 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 37 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1


 99%|█████████▊| 1121/1137 [1:18:23<00:34,  2.17s/it]

Rate limit exceeded. Retrying in 34 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 34 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 2 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\n

100%|██████████| 1137/1137 [1:20:30<00:00,  4.25s/it]

   post_index most_similar_post_index similarity_score
0     2001794                 2001557                1
1    20001003                 2001794                2
2     2001015                20001003                7
3     2000505                20001003                6


In [108]:
similarity_ext.set_index(similarity_ext['post_index'], inplace=True)

In [ ]:
similarity_ext.join(YT_claim_embeds_sort.set_index('Mark_ID')['post_expansion_gpt4o'], rsuffix='_target')

In [110]:
similarity_ext = similarity_ext.join(YT_claim_embeds_sort.set_index('Mark_ID')['post_expansion_gpt4o'], rsuffix='_target')

In [111]:
def convert_to_int(value):
    if isinstance(value, list):
        return np.nan
    try:
        return int(value)
    except ValueError:
        return np.nan

similarity_ext['most_similar_post_index'].apply(convert_to_int).isna().sum()

In [113]:
#apparently there were some lists in the most_similar_post_index column, which need to be converted to NaNs
similarity_ext.loc[:, 'most_similar_post_index_int'] = similarity_ext['most_similar_post_index'].apply(convert_to_int)

In [114]:
similarity_ext.loc[similarity_ext['most_similar_post_index_int'].isna(), :] 

,post_index,most_similar_post_index,similarity_score,post_expansion_gpt4o,most_similar_post_index_int
post_index,,,,,
2001038,2001038,[],[],[],NaN
2000066,2000066,[],[],[],NaN
2001530,2001530,[],[],[],NaN


In [ ]:
#apparently these lists are empty lists, which can be converted to NaNs

In [ ]:
similarity_ext.join(YT_claim_embeds_sort.set_index('Mark_ID')['commentText'], rsuffix='_target').merge(YT_claim_embeds_sort.loc[:, ['Mark_ID', 'post_expansion_gpt4o']], left_on = 'most_similar_post_index_int', right_on = 'Mark_ID' , suffixes=('', '_source'))


In [ ]:
similarity_ext.merge(YT_claim_embeds_sort.loc[:, ['Mark_ID', 'post_expansion_gpt4o']], left_on = 'most_similar_post_index_int', right_on = 'Mark_ID' , suffixes=('', '_source'))


In [117]:
similarity_ext = similarity_ext.merge(YT_claim_embeds_sort.loc[:, ['Mark_ID', 'post_expansion_gpt4o']], left_on = 'most_similar_post_index_int', right_on = 'Mark_ID' , suffixes=('', '_source'))


In [118]:
similarity_ext.drop(columns=['Mark_ID'], inplace=True)

In [119]:
similarity_ext.rename(columns={'most_similar_post_index_int': 'most_similar_ext_ID_gpt4o', 'similarity_score': 'similarity_score_ext_gpt4o', 'post_expansion_gpt4o_source':'post_expenasion_most_sim_gpt4o' }, inplace=True)


In [120]:
similarity_ext.loc[similarity_ext.post_index == 119, ['post_index', 'most_similar_ext_ID_gpt4o', 'similarity_score_ext_gpt4o', 'post_expenasion_most_sim_gpt4o']]

,post_index,most_similar_ext_ID_gpt4o,similarity_score_ext_gpt4o,post_expenasion_most_sim_gpt4o


In [121]:
similarity_ext.head()

,post_index,most_similar_post_index,similarity_score_ext_gpt4o,post_expansion_gpt4o,most_similar_ext_ID_gpt4o,post_expenasion_most_sim_gpt4o
0,2001794,2001557,1,Stop killing inocent people!,2001557.0,Lux you’re welcome
1,20001003,2001794,2,"The Syrian crisis is a horrible thing, but has anyone noticed the situation’s strange components...",2001794.0,Stop killing inocent people!
2,2001015,20001003,7,"But the environment and everything that humans are doing, like the conflicts in Syria and the in...",20001003.0,"The Syrian crisis is a horrible thing, but has anyone noticed the situation’s strange components..."
3,2000505,20001003,6,1. Why is the rest of the world praising the US? Why is the rest of the world (Britain and Franc...,20001003.0,"The Syrian crisis is a horrible thing, but has anyone noticed the situation’s strange components..."
4,2001417,20001003,5,It would have cost less to just fly a B-52 over the targets and carpet bomb the living daylights...,20001003.0,"The Syrian crisis is a horrible thing, but has anyone noticed the situation’s strange components..."


In [122]:
YT_claim_embeds = YT_claim_embeds.merge(similarity_ext.loc[:, ['post_index', 'most_similar_ext_ID_gpt4o', 'similarity_score_ext_gpt4o', 'post_expenasion_most_sim_gpt4o']], how='left', left_on='Mark_ID', right_on='post_index', suffixes=('', '_ext_gpt4o'))

In [ ]:
YT_claim_embeds.loc[:, ['Mark_ID', 'commentText', 'post_expansion_gpt4o', 'most_similar_ext_ID_gpt4o', 'similarity_score_ext_gpt4o', 'post_expenasion_most_sim_gpt4o']]


In [ ]:
YT_claim_embeds.loc[:, ['commentText', 'post_expansion_gpt4o']]

In [125]:
#save data:
YT_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere_YT_posts.parquet')

In [130]:
dataset_claim_embeds = dataset_claim_embeds.merge(YT_claim_embeds.loc[:, ['Mark_ID', 'most_similar_ext_ID_gpt4o', 'similarity_score_ext_gpt4o', 'post_expenasion_most_sim_gpt4o']], how='left', left_on='Mark_ID', right_on='Mark_ID', suffixes=('', '_ext_gpt4o'))

In [132]:
#save data:
dataset_claim_embeds.to_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')

In [9]:
#load data:
YT_claim_embeds = pd.read_parquet(f'{CFG.report_dir}/pubsphere_YT_posts.parquet')

In [13]:
YT_claim_embeds['claims_gpt4o'].str.split('",').explode()


0                                                                           []
1                                                 ["Minerals have rights too"]
2       ["The death of free and civil dialogue has been effectively depicted"]
3                                                 ["No-one else will hug him"]
4                                                                           []
                                         ...                                  
3131                           "Putin is aligned with al Assad to combat Daesh
3131                                             "The U.S. is against al Assad
3131                                "Russia and the U.S. are both against ISIS
3131                             "Russia is aligned with the Syrian government
3131      "The U.S. is against the Syrian government for attacking civilians"]
Name: claims_gpt4o, Length: 6211, dtype: object

In [149]:
#create a expanded dataset of claims along with Mark_IDs and videoTitles:
YT_claim_embeds['claims_split'] = YT_claim_embeds['claims_gpt4o'].str.split('",')
YT_claim_embeds_explode = YT_claim_embeds.explode('claims_split')
YT_claim_embeds_explode = YT_claim_embeds_explode.reset_index(drop=True)
YT_claim_embeds_explode['claims_split'] = YT_claim_embeds_explode['claims_split'].str.replace('"', '').str.replace('[', '').str.replace(']', '').str.strip()

In [ ]:
YT_claim_embeds_explode

In [ ]:
YT_claim_embeds_explode.sort_values(by='Time_comment_dt', ascending=True).reset_index(drop=False).head().loc[:, ['Mark_ID', 'videoTitle', 'claims_split', 'Time_comment_dt']]

In [152]:
YT_claim_embeds_explode.loc[:, "explode_index"] = YT_claim_embeds_explode.index

In [33]:
###############################
#DO NO REUSE THIS CELL, IT MISSES A GOOD claim_split-level identifier and needs to be updated in line with the simscore annotation for expanded claims as described below
###############################

#get similarity scores for claims using GPT4o:
#get simscore for posts using GPT4o:

chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
YT_input_sort = YT_claim_embeds_explode.sort_values(by='Time_comment_dt', ascending=True).reset_index(drop=False)
groupeddata = YT_input_sort.groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            retry_count = 0
            max_retries = 10
    
            while retry_count < max_retries:
                try:  
                    response = requests.post(
                            url=api_endpoint,
                            headers=headers,
                            json={
                                'model': MODELgpt4o,
                                'messages': [
                                    {
                                        "role": "system",
                                        "content": SYSTEM_claim_simscore                        
                                    },
                                    {
                                        "role": "user",
                                        "content": f'"Target claim":<<{row["claims_split"]}>>, "preceding claims":\n<{df.loc[:,['Mark_ID', 'claims_split']][:index].to_json(orient='records')}>'
                                    
                                    }
                                ],
                                'temperature': temperature_0,  
                                'seed': SEED
                            }
                        )
                    
                    if response.status_code == 200:
                        data_response = response.json()
                        # Check if 'response' key exists in the JSON response
                        if 'choices' in data_response:
                            content = data_response["choices"][0]["message"]["content"]
                            # Remove code block markers if present
                            content = re.sub(r'```json|```', '', content).strip()
                            try:
                                content = json.loads(content)   
                                # Check if content_json is a list or a dictionary
                                if isinstance(content, dict):
                                    chunked_result.append({
                                        "post_index": row['Mark_ID'],
                                        "most_similar_claim_index": content.get('most_similar_claim_index', ''),
                                        'similarity_score': content.get('similarity_score', '')  
                                    })
                                elif isinstance(content, list):
                                    for item in content:
                                        chunked_result.append({
                                            "post_index": row['Mark_ID'],
                                            "most_similar_claim_index": item.get('most_similar_claim_index', ''),
                                            'similarity_score': item.get('similarity_score', '')  
                                        })
                                else:
                                    print(f"Unexpected content type: {type(content)}")

                            except json.JSONDecodeError as e:
                                print(f"Error decoding JSON: {e}")
                                print(data_response)
                        else:
                            print(f"No 'choices' key in response or 'choices' is empty: {content}")                        
                        break  # Exit the retry loop on success  
                    elif response.status_code == 429:
                        retry_count += 1
                        retry_after = response.headers.get("Retry-After")
                        error_message = response.json().get("error", {}).get("message", "")
                        retry_after = 30  # Default to 30 seconds if not found
                    
                        # Extract retry time from the error message
                        if "Try again in" in error_message:
                            match = re.search(r"Try again in (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                        elif "Please retry after" in error_message:
                            match = re.search(r"Please retry after (\d+) second", error_message)
                            if match:
                                try:
                                    retry_after = int(match.group(1))
                                except (IndexError, ValueError) as e:
                                    print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                    pass
                                
                                
                        else:
                            retry_after = 30  # Default to 30 seconds if not found
                            print(f"Rate limit exceeded. Defaulting to retry in {retry_after} seconds.")
                                    
                
                        print(f"Rate limit exceeded. Retrying in {retry_after} seconds: {response.json()}. Retry count = {retry_count}") 
                        time.sleep(retry_after)
                        
                    #    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                    #    print(response.text)
                    #    time.sleep(wait_time)
                    elif response.status_code == 500:
                        retry_count += 1
                        wait_time = 20
                        print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to connect to API. Status code: {response.status_code}")
                        print(response.text)
                        break
                except requests.exceptions.RequestException as e:   
                    print(f"Failed to connect to API: {e}")
                    retry_count += 1
                    wait_time = 60
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)   
        
                if retry_count >= max_retries:
                    print(f"Max retries reached for index {index}. Skipping to next item.")
                    break  # Exit the loop if max retries are reached              

# Convert the list of dictionaries to a DataFrame
similarity_claim = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(similarity_claim.head(4))


  0%|          | 3/1137 [00:25<2:42:23,  8.59s/it]

Rate limit exceeded. Retrying in 32 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 32 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1


  1%|          | 10/1137 [01:14<1:33:14,  4.96s/it]

Rate limit exceeded. Retrying in 10 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 10 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 1 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nR

  1%|          | 13/1137 [03:28<6:47:13, 21.74s/it]

...

Rate limit exceeded. Retrying in 19 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 19 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 6 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 6 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\n

100%|█████████▉| 1135/1137 [3:35:07<00:05,  2.57s/it]

Rate limit exceeded. Retrying in 27 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 27 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 2 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\n

100%|██████████| 1137/1137 [3:37:55<00:00, 11.50s/it]

   post_index most_similar_claim_index similarity_score
0     2001794                  2001557                1
1    20001003                  2001794                5
2    20001003                        3                5
3    20001003                 20001003                7


In [165]:
similarity_claim = pd.DataFrame(chunked_result)

In [78]:
similarity_claim

,post_index,most_similar_claim_index,similarity_score
0,2001794,2001557,1
1,20001003,2001794,5
2,20001003,3,5
3,20001003,20001003,7
4,20001003,3,5
...,...,...,...
4847,2001730,2001590,6
4848,2001730,2001590,5
4849,2001730,2002436,7
4850,2001730,2002436,6


In [92]:
similarity_claim.most_similar_claim_index.value_counts()

most_similar_claim_index
[]         542
1          392
0          374
3          210
2          187
          ... 
48539        1
2001747      1
125915       1
125904       1
2002196      1
Name: count, Length: 1445, dtype: int64

In [87]:
similarity_claim.post_index.value_counts()

post_index
2000933    14
2000939    14
2002432    14
2001304    12
2001040    12
           ..
2001577     1
2001806     1
2001150     1
2002043     1
2001820     1
Name: count, Length: 2206, dtype: int64

In [97]:
#produce dataframe for context:
chunked_result_df: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
YT_input_sort = YT_claim_embeds_explode.sort_values(by='Time_comment_dt', ascending=True).reset_index(drop=False)
groupeddata = YT_input_sort.groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df.loc[:, 'preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            chunked_result_df.append({"post_index": index, "Mark_ID": row['Mark_ID'], 'explode_ID': row['explode_index'], 'claims_split': row['claims_split'], 'Time_comment_dt': row['Time_comment_dt']})

frame = pd.DataFrame(chunked_result_df)
print(frame)

100%|██████████| 1137/1137 [00:00<00:00, 2168.56it/s]

      post_index   Mark_ID  explode_ID  \
0              1   2001794        4724   
1              2  20001003        6204   
2              3  20001003        6205   
3              4  20001003        6206   
4              5  20001003        6207   
...          ...       ...         ...   
5069          44   2001730        4572   
5070          45   2001730        4576   
5071          46   2001730        4574   
5072          47   2001730        4573   
5073          48   2001046        3129   

                                         claims_split  \
0                        Stop killing innocent people   
1               The Syrian crisis is a horrible thing   
2            Al Assad is against Daesh and the rebels   
3      Putin is aligned with al Assad to combat Daesh   
4                        The U.S. is against al Assad   
...                                               ...   
5069      Celebrities are used to brainwash Americans   
5070                                Lib

In [140]:
not_in_frame = similarity_claim[~similarity_claim['post_index'].isin(frame['Mark_ID'])]
print("Values in similarity_claim.post_index not in frame.post_index:")
print(not_in_frame)

# Identify values in frame.post_index that are not in similarity_claim.post_index
not_in_similarity_claim = frame[~frame['Mark_ID'].isin(similarity_claim['post_index'])]
print("\nValues in frame.post_index not in similarity_claim.post_index:")
print(not_in_similarity_claim)

# Find common values in both post_index columns
common_values = similarity_claim[similarity_claim['post_index'].isin(frame['Mark_ID'])]
print("\nCommon values in both post_index columns:")
print(common_values)

Values in similarity_claim.post_index not in frame.post_index:
Empty DataFrame
Columns: [post_index, most_similar_claim_index, similarity_score, index]
Index: []

Values in frame.post_index not in similarity_claim.post_index:
      post_index  Mark_ID  explode_ID claims_split           Time_comment_dt  \
14            15  2000953        2936              2018-04-15 02:32:43+00:00   
45            13   200225        1127              2018-08-21 13:50:07+00:00   
58             4  2001021        3081              2019-05-08 12:20:10+00:00   
84            30  2001332        3750              2019-05-15 10:57:37+00:00   
116           15  2001391        3872              2019-06-22 15:30:37+00:00   
...          ...      ...         ...          ...                       ...   
4957           7  2001744        4610              2019-01-19 16:05:47+00:00   
4979           8  2001379        3847              2019-06-17 16:26:14+00:00   
4984          13  2001531        4167              201

In [141]:
#common values appears to show the link with the output results:
common_values = common_values.join(frame['explode_ID'])


In [142]:
common_values

,post_index,most_similar_claim_index,similarity_score,index,explode_ID
0,2001794,2001557,1,0,4724
1,20001003,2001794,5,1,6204
2,20001003,3,5,2,6205
3,20001003,20001003,7,3,6206
4,20001003,3,5,4,6207
...,...,...,...,...,...
4847,2001730,2001590,6,4847,4612
4848,2001730,2001590,5,4848,4614
4849,2001730,2002436,7,4849,4613
4850,2001730,2002436,6,4850,1205


In [143]:
#similarity_claim = similarity_claim.join(YT_input_sort.set_index('dataset_index')['commentText'], rsuffix='_target')
common_values = common_values.merge(YT_input_sort.loc[:, ['commentText','claims_split', 'explode_index']], left_on='explode_ID', right_on='explode_index', suffixes=('', '_target'))

In [144]:
def convert_to_int(value):
    if isinstance(value, list):
        return np.nan
    try:
        return int(value)
    except ValueError:
        return np.nan

common_values.loc[:, 'most_similar_claim_index_int'] = common_values.loc[:, 'most_similar_claim_index'].apply(convert_to_int)

In [145]:
common_values.merge(YT_claim_embeds.loc[:, ['Mark_ID', 'commentText']], left_on = 'most_similar_claim_index_int', right_on = 'Mark_ID' , suffixes=('', '_source'), how='left')

,post_index,most_similar_claim_index,similarity_score,index,explode_ID,commentText,claims_split,explode_index,most_similar_claim_index_int,Mark_ID,commentText_source
0,2001794,2001557,1,0,4724,Stop killing inocent people!,Stop killing innocent people,4724,2001557.0,2001557.0,Lux you’re welcome
1,20001003,2001794,5,1,6204,"The Syrian crisis is a horrible thing, but has anyone noticed the situation’s strange components...",The Syrian crisis is a horrible thing,6204,2001794.0,2001794.0,Stop killing inocent people!
2,20001003,3,5,2,6205,"The Syrian crisis is a horrible thing, but has anyone noticed the situation’s strange components...",Al Assad is against Daesh and the rebels,6205,3.0,NaN,NaN
3,20001003,20001003,7,3,6206,"The Syrian crisis is a horrible thing, but has anyone noticed the situation’s strange components...",Putin is aligned with al Assad to combat Daesh,6206,20001003.0,20001003.0,"The Syrian crisis is a horrible thing, but has anyone noticed the situation’s strange components..."
4,20001003,3,5,4,6207,"The Syrian crisis is a horrible thing, but has anyone noticed the situation’s strange components...",The U.S. is against al Assad,6207,3.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4847,2001730,2001590,6,4847,4612,Will Roberts you will see what a clown you are. Your comments are useless garbage. None of us kn...,None of us know what any of these agencies have,4612,2001590.0,2001590.0,"Again, I reiterate, it is the Taliban's fault. The war could have been avoided if they took him ..."
4848,2001730,2001590,5,4848,4614,Will Roberts you will see what a clown you are. Your comments are useless garbage. None of us kn...,It is laughable to think Trump is innocent of all,4614,2001590.0,2001590.0,"Again, I reiterate, it is the Taliban's fault. The war could have been avoided if they took him ..."
4849,2001730,2002436,7,4849,4613,Will Roberts you will see what a clown you are. Your comments are useless garbage. None of us kn...,Many people around Trump are crooked,4613,2002436.0,2002436.0,"Well I apologize for not practicing the best grammar online, especially since my it's not even t..."
4850,2001730,2002436,6,4850,1205,He is only a coffee boy lawyer.,He is only a coffee boy lawyer,1205,2002436.0,2002436.0,"Well I apologize for not practicing the best grammar online, especially since my it's not even t..."


In [146]:
common_values = common_values.merge(YT_claim_embeds.loc[:, ['Mark_ID', 'commentText']], left_on = 'most_similar_claim_index_int', right_on = 'Mark_ID' , suffixes=('', '_source'), how='left')
common_values.drop(columns=['Mark_ID'], inplace=True)
common_values.rename(columns={'most_similar_claim_index': 'most_similar_claim_index_gpt4o', 'most_similar_claim_index_int': 'most_similar_claim_index_int_gpt4o', 'similarity_score': 'similarity_score_claim_gpt4o', 'commentText':'commentText_most_sim_claim_gpt4o' }, inplace=True)


In [ ]:
common_values

In [153]:
YT_claim_embeds_explode = YT_claim_embeds_explode.merge(common_values.loc[:, ['most_similar_claim_index_int_gpt4o', 'similarity_score_claim_gpt4o', 'commentText_most_sim_claim_gpt4o', 'explode_ID']], left_on = 'explode_index', right_on = 'explode_ID' , suffixes=('', '_source'), how='left')
YT_claim_embeds_explode.drop(columns=['explode_ID'], inplace=True)

In [ ]:
YT_claim_embeds_explode['similarity_score_claim_gpt4o'] = YT_claim_embeds_explode['similarity_score_claim_gpt4o'].apply(convert_to_int)

In [164]:
#save data:
YT_claim_embeds_explode.to_parquet(f'{CFG.report_dir}/pubsphere_YT_post_claim_explode.parquet')

In [7]:
#load data:
YT_claim_embeds_explode = pd.read_parquet(f'{CFG.report_dir}/pubsphere_YT_post_claim_explode.parquet')


In [ ]:
YT_claim_embeds_explode.loc[:, ['Mark_ID', 'videoTitle', 'claims_split', 'Time_comment_dt', 'most_similar_claim_index_int_gpt4o', 'similarity_score_claim_gpt4o', 'commentText_most_sim_claim_gpt4o']]


In [13]:
YT_claim_embeds_explode.videoTitle.value_counts()

videoTitle
John Berman: Is it even news when the President lies?                                                  50
Rep. Adam Schiff: Authoritarianism vs. Democracy  | Real Time with Bill Maher (HBO)                    50
iTunes, Assad, and Right Said Fred: Last Week Tonight with John Oliver (HBO)                           49
'Hannity' panel on the important questions Mueller needs to answer                                     46
Trump talks race, football, foreign policy and more ahead of the Super Bowl                            46
                                                                                                       ..
2020 Democrats Divided Over Impeachment After Mueller Report | NBC Nightly News                         1
Senator Lindsey Graham: Why He's Opposed to the Iran Nuclear Deal - Late Night with Seth Meyers         1
Matthews: How Does Pres. Donald Trump Justify Putting More Troops In Harm's Way? | Hardball | MSNBC     1
ISIS-destroyed monuments reconstruc

In [22]:
YT_claim_embeds_explode['videoTitle'].iloc[2:4].to_list()

["Fmr. Prosecutor On Donald Trump Mocking Kavanaugh Accuser: 'Who Are We?' | The 11th Hour | MSNBC",
 'Nicolle Wallace: John McCain In Death Was Able To Shame Donald Trump In Life | The 11th Hour | MSNBC']

In [23]:
YT_claim_embeds_explode.loc[YT_claim_embeds_explode['videoTitle'] == 'Nicolle Wallace: John McCain In Death Was Able To Shame Donald Trump In Life | The 11th Hour | MSNBC',:]




,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,claims_ext_gpt4o,post_index_ext_gpt4o,most_similar_ext_ID_gpt4o,similarity_score_ext_gpt4o,post_expenasion_most_sim_gpt4o,claims_split,explode_index,most_similar_claim_index_int_gpt4o,similarity_score_claim_gpt4o,commentText_most_sim_claim_gpt4o
3,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,"[""No-one else will hug him""]",NaN,NaN,None,None,No-one else will hug him,3,NaN,NaN,None


In [ ]:
#the missing values in the similarity scores appear to be due to single comment videos, which do not have any preceding comments to compare with.

In [25]:
#load data:
YT_claim_embeds = pd.read_parquet(f'{CFG.report_dir}/pubsphere_YT_posts.parquet')

In [26]:
YT_claim_embeds.columns.to_list()

['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'commentText',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',


In [59]:
#add ascending group index per comment:
YT_claim_embeds_sort = YT_claim_embeds.sort_values(by='Time_comment_dt', ascending=True).reset_index(drop=True)
YT_claim_embeds_sort['ascending_comment_index'] = YT_claim_embeds_sort.groupby('videoTitle').cumcount()
YT_claim_embeds_sort['preceding_comment_index'] = YT_claim_embeds_sort.groupby('videoTitle')['ascending_comment_index'].shift(1)

In [90]:
YT_claim_embeds_sort.sort_values(by='Time_comment_dt', ascending=True).loc[:, ['videoTitle', 'Time_comment_dt', 'ascending_comment_index', 'preceding_comment_index']].groupby('videoTitle').value_counts()

videoTitle                                                                    Time_comment_dt            ascending_comment_index  preceding_comment_index
"Haley: Airstrikes ""crippled"" Syria’s chemical weapons program  "           2018-04-15 01:47:04+00:00  1                        0.0                        1
                                                                              2018-04-15 01:48:53+00:00  2                        1.0                        1
                                                                              2018-04-15 01:52:34+00:00  3                        2.0                        1
                                                                              2018-04-15 02:07:45+00:00  4                        3.0                        1
                                                                              2018-04-15 02:28:07+00:00  5                        4.0                        1
                                                   

In [62]:
YT_claim_embeds = YT_claim_embeds.merge(YT_claim_embeds_sort.loc[:, ['Mark_ID', 'ascending_comment_index', 'preceding_comment_index']], on='Mark_ID', suffixes=('', '_sort'), how='left')

In [63]:
#get simscore for expanded claims using GPT4o:
#first explode the expanded claims:
#create a expanded dataset of claims along with Mark_IDs and videoTitles:

YT_claim_embeds['claims_exp_split'] = YT_claim_embeds['claims_ext_gpt4o'].str.split('",')
YT_claim_embeds_exp_explode = YT_claim_embeds.explode('claims_exp_split')
YT_claim_embeds_exp_explode = YT_claim_embeds_exp_explode.reset_index(drop=True)
YT_claim_embeds_exp_explode['claims_exp_split'] = YT_claim_embeds_exp_explode['claims_exp_split'].str.replace('"', '').str.replace('[', '').str.replace(']', '').str.strip()

In [113]:
YT_claim_embeds_exp_explode

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,most_similar_ext_ID_gpt4o,similarity_score_ext_gpt4o,post_expenasion_most_sim_gpt4o,claims_exp_split,ascending_comment_index_sort,preceding_comment_index_sort,ascending_comment_index,preceding_comment_index,exp_explode_index,claim_per_video_index
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,NaN,None,None,,641,640.0,0,NaN,0,0
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,2000336.0,3,@Zquadfather you sound like an Antifa punk!!!,Minerals have rights too,2927,2926.0,19,18.0,1,0
2,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,NaN,None,None,The death of free and civil dialogue has been effectively depicted,1453,1452.0,0,NaN,2,0
3,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,NaN,None,None,No-one else will hug him,1378,1377.0,0,NaN,3,1
4,11/30/2021 17:24:18,11/30/2021 17:27:16,213.127.109.191,1,6,Ugw2eTvkZLfH9MDVg1R4AaABAg,1214,1,2,1,...,2000601.0,1,I RESPECTFULLY want an UPDATE ON THE DRAIN THE SWAMP PROMISE. You Tube killed my “Q” subscriptio...,,2966,2965.0,1,0.0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6293,11/19/2021 17:49:17,11/19/2021 17:51:04,213.127.109.191,1,6,UgwPOHIDyICm10k0Mvx4AaABAg,20001003,0,3,1,...,2001794.0,2,Stop killing inocent people!,Putin is aligned with al Assad to combat Daesh,926,925.0,2,1.0,6293,27
6294,11/19/2021 17:49:17,11/19/2021 17:51:04,213.127.109.191,1,6,UgwPOHIDyICm10k0Mvx4AaABAg,20001003,0,3,1,...,2001794.0,2,Stop killing inocent people!,The U.S. is against al Assad,926,925.0,2,1.0,6294,28
6295,11/19/2021 17:49:17,11/19/2021 17:51:04,213.127.109.191,1,6,UgwPOHIDyICm10k0Mvx4AaABAg,20001003,0,3,1,...,2001794.0,2,Stop killing inocent people!,Russia and the U.S. are both against ISIS,926,925.0,2,1.0,6295,0
6296,11/19/2021 17:49:17,11/19/2021 17:51:04,213.127.109.191,1,6,UgwPOHIDyICm10k0Mvx4AaABAg,20001003,0,3,1,...,2001794.0,2,Stop killing inocent people!,Russia is aligned with the Syrian government,926,925.0,2,1.0,6296,2


In [64]:
#add an exp_explode_index column:
YT_claim_embeds_exp_explode.loc[:, "exp_explode_index"] = YT_claim_embeds_exp_explode.index

In [65]:
YT_claim_embeds_exp_explode.loc[:, 'claim_per_video_index'] = YT_claim_embeds_exp_explode.sort_values(by='Time_comment_dt', ascending=True).reset_index(drop=False).groupby('videoTitle').cumcount()

In [96]:
YT_claim_embeds_exp_explode.loc[YT_claim_embeds_exp_explode.claims_exp_split=='',:].count()

StartDate                       1291
RecordedDate                    1291
IPAddress                       1291
Finished                        1291
Coder                           1291
                                ... 
preceding_comment_index_sort    1290
ascending_comment_index         1291
preceding_comment_index          742
exp_explode_index               1291
claim_per_video_index           1291
Length: 135, dtype: int64

In [107]:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
YT_input_sort = YT_claim_embeds_exp_explode.sort_values(by=['Time_comment_dt','ascending_comment_index'], ascending=True).reset_index(drop=False)[50:60]
groupeddata = YT_input_sort.groupby(GROUPER)

for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df = df.loc[df.claims_exp_split != '',:]
    df.sort_values(by=['Time_comment_dt','ascending_comment_index'], ascending=True).reset_index(drop=False)
    print('dataframe:')
    print(df.loc[:,['exp_explode_index', 'claims_exp_split', 'ascending_comment_index']])
    for index, row in df.iterrows():
        print('loop index:')
        print(index)
        if pd.notna(row['preceding_comment_index']):
            if row['claims_exp_split'] != '':
                print(f'"Target claim":<<{row["claims_exp_split"]}>>, "preceding claims":\n<{df.loc[df.ascending_comment_index != row['ascending_comment_index'], ['exp_explode_index', 'claims_exp_split']].iloc[:index].to_json(orient='records')}>')
                print(row['exp_explode_index'])
                print(row['preceding_comment_index'])
                print(df.loc[:,['exp_explode_index', 'claims_exp_split']].iloc[:int(row['preceding_comment_index'])])
            

100%|██████████| 5/5 [00:00<00:00, 386.49it/s]

dataframe:
Empty DataFrame
Columns: [exp_explode_index, claims_exp_split, ascending_comment_index]
Index: []
dataframe:
Empty DataFrame
Columns: [exp_explode_index, claims_exp_split, ascending_comment_index]
Index: []
dataframe:
Empty DataFrame
Columns: [exp_explode_index, claims_exp_split, ascending_comment_index]
Index: []
dataframe:
   exp_explode_index                          claims_exp_split  \
0               3013  Using a broad brush can cost credibility   

   ascending_comment_index  
0                        0  
loop index:
0
dataframe:
   exp_explode_index  \
0               2612   
4               6171   
5               6172   

                                              claims_exp_split  \
0                 Over 9 million Syrians have fled the country   
4  The guitar player is not actually playing in the background   
5                              Only bass is heard in the track   

   ascending_comment_index  
0                        0  
4                        4

In [109]:
#get similarity scores for claims using GPT4o:
#adjusted request to compare claims only with preceding claims in previous comments, so not preceding claims within the same comment:
#get simscore for posts using GPT4o:

chunked_result: typing.List[pd.DataFrame] = []
GROUPER='videoTitle'
#sort data by Time_comment
YT_input_sort = YT_claim_embeds_exp_explode.sort_values(by=['Time_comment_dt','ascending_comment_index'], ascending=True).reset_index(drop=False)
groupeddata = YT_input_sort.groupby(GROUPER)

for group, df in tqdm.tqdm(groupeddata):
    df.reset_index(drop=True, inplace=True)
    df = df.loc[df.claims_exp_split != '',:]
    df.sort_values(by=['Time_comment_dt','ascending_comment_index'], ascending=True).reset_index(drop=False)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_comment_index']):
            if row['claims_exp_split'] != '':
                retry_count = 0
                max_retries = 10

                while retry_count < max_retries:
                    try:  
                        response = requests.post(
                                url=api_endpoint,
                                headers=headers,
                                json={
                                    'model': MODELgpt4o,
                                    'messages': [
                                        {
                                            "role": "system",
                                            "content": SYSTEM_claim_simscore                        
                                        },
                                        {
                                            "role": "user",
                                            "content": f'"Target claim":<<{row["claims_exp_split"]}>>, "preceding claims":\n<{df.loc[df.ascending_comment_index != row['ascending_comment_index'], ['exp_explode_index', 'claims_exp_split']].iloc[:index].to_json(orient='records')}>'
                                        }
                                    ],
                                    'temperature': temperature_0,  
                                    'seed': SEED
                                }
                            )

                        if response.status_code == 200:
                            data_response = response.json()
                            # Check if 'response' key exists in the JSON response
                            if 'choices' in data_response:
                                content = data_response["choices"][0]["message"]["content"]
                                # Remove code block markers if present
                                content = re.sub(r'```json|```', '', content).strip()
                                try:
                                    content = json.loads(content)   
                                    # Check if content_json is a list or a dictionary
                                    if isinstance(content, dict):
                                        chunked_result.append({
                                            "claim_exp_index": row['exp_explode_index'],
                                            'claim_per_video_index': row['claim_per_video_index'],
                                            "most_similar_claim_index": content.get('most_similar_claim_index', ''),
                                            'similarity_score': content.get('similarity_score', '')  
                                        })
                                    elif isinstance(content, list):
                                        for item in content:
                                            chunked_result.append({
                                            "claim_exp_index": row['exp_explode_index'],
                                            'claim_per_video_index': row['claim_per_video_index'],
                                            "most_similar_claim_index": item.get('most_similar_claim_index', ''),
                                            'similarity_score': item.get('similarity_score', '') 
                                            })
                                    else:
                                        print(f"Unexpected content type: {type(content)}")

                                except json.JSONDecodeError as e:
                                    print(f"Error decoding JSON: {e}")
                                    print(data_response)
                            else:
                                print(f"No 'choices' key in response or 'choices' is empty: {content}")                        
                            break  # Exit the retry loop on success  
                        elif response.status_code == 429:
                            retry_count += 1
                            retry_after = response.headers.get("Retry-After")
                            error_message = response.json().get("error", {}).get("message", "")
                            retry_after = 30  # Default to 30 seconds if not found

                            # Extract retry time from the error message
                            if "Try again in" in error_message:
                                match = re.search(r"Try again in (\d+) second", error_message)
                                if match:
                                    try:
                                        retry_after = int(match.group(1))
                                    except (IndexError, ValueError) as e:
                                        print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                        pass
                            elif "Please retry after" in error_message:
                                match = re.search(r"Please retry after (\d+) second", error_message)
                                if match:
                                    try:
                                        retry_after = int(match.group(1))
                                    except (IndexError, ValueError) as e:
                                        print(f"Rate limit exceeded. Error extracting retry time: {e}. Retrying in {retry_after} seconds.")
                                        pass
                                    
                                    
                            else:
                                retry_after = 30  # Default to 30 seconds if not found
                                print(f"Rate limit exceeded. Defaulting to retry in {retry_after} seconds.")


                            print(f"Rate limit exceeded. Retrying in {retry_after} seconds: {response.json()}. Retry count = {retry_count}") 
                            time.sleep(retry_after)

                        #    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                        #    print(response.text)
                        #    time.sleep(wait_time)
                        elif response.status_code == 500:
                            retry_count += 1
                            wait_time = 20
                            print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                            print(response.text)
                            time.sleep(wait_time)
                        else:
                            print(f"Failed to connect to API. Status code: {response.status_code}")
                            print(response.text)
                            break
                    except requests.exceptions.RequestException as e:   
                        print(f"Failed to connect to API: {e}")
                        retry_count += 1
                        wait_time = 60
                        print(f"Retrying in {wait_time} seconds...")
                        time.sleep(wait_time)   

                    if retry_count >= max_retries:
                        print(f"Max retries reached for index {index}. Skipping to next item.")
                        break  # Exit the loop if max retries are reached              

# Convert the list of dictionaries to a DataFrame
similarity_claim_exp = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(similarity_claim_exp.head(4))

  0%|          | 0/1137 [00:00<?, ?it/s]

Rate limit exceeded. Retrying in 4 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 4 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 1 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nRec

  0%|          | 4/1137 [01:43<7:30:23, 23.85s/it] 

Rate limit exceeded. Retrying in 23 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 23 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 2 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\n

  1%|          | 12/1137 [03:48<5:26:14, 17.40s/it]

...

Rate limit exceeded. Retrying in 14 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 14 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 2 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\n

 99%|█████████▉| 1130/1137 [2:40:50<01:07,  9.69s/it]

Rate limit exceeded. Retrying in 29 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 29 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4o-2024-08-06\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '429'}}. Retry count = 1
Rate limit exceeded. Retrying in 4 seconds: {'error': {'message': 'litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 4 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\n

100%|██████████| 1137/1137 [2:42:58<00:00,  8.60s/it]

   claim_exp_index  claim_per_video_index most_similar_claim_index  \
0             4782                      3                     6291   
1             6291                     25                     4782   
2             6292                     26                     3082   
3             6293                     27                     3082   

  similarity_score  
0                5  
1                5  
2                5  
3                5  


In [72]:
YT_claim_embeds_exp_explode.columns.to_list()   


['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'commentText',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',


In [110]:
similarity_claim_exp.merge(YT_claim_embeds_exp_explode.loc[:, ['Mark_ID', 'claims_ext_gpt4o', 'claims_exp_split', 'ascending_comment_index', 'preceding_comment_index', 'claim_per_video_index', 'Time_comment_dt', 'exp_explode_index']], left_on='claim_exp_index', right_on='exp_explode_index', suffixes=('', '_source'))

,claim_exp_index,claim_per_video_index,most_similar_claim_index,similarity_score,Mark_ID,claims_ext_gpt4o,claims_exp_split,ascending_comment_index,preceding_comment_index,claim_per_video_index_source,Time_comment_dt,exp_explode_index
0,4782,3,6291,5,2001794,"[""Stop killing innocent people""]",Stop killing innocent people,1,0.0,3,2018-04-15 01:47:04+00:00,4782
1,6291,25,4782,5,20001003,"[""The Syrian crisis is a horrible thing"", ""Al Assad is against Daesh and the rebels"", ""Putin is ...",The Syrian crisis is a horrible thing,2,1.0,25,2018-04-15 01:48:53+00:00,6291
2,6292,26,3082,5,20001003,"[""The Syrian crisis is a horrible thing"", ""Al Assad is against Daesh and the rebels"", ""Putin is ...",Al Assad is against Daesh and the rebels,2,1.0,26,2018-04-15 01:48:53+00:00,6292
3,6293,27,3082,5,20001003,"[""The Syrian crisis is a horrible thing"", ""Al Assad is against Daesh and the rebels"", ""Putin is ...",Putin is aligned with al Assad to combat Daesh,2,1.0,27,2018-04-15 01:48:53+00:00,6293
4,6294,28,3082,5,20001003,"[""The Syrian crisis is a horrible thing"", ""Al Assad is against Daesh and the rebels"", ""Putin is ...",The U.S. is against al Assad,2,1.0,28,2018-04-15 01:48:53+00:00,6294
...,...,...,...,...,...,...,...,...,...,...,...,...
3326,4630,0,2223,6,2001730,"[""John Oliver is using celebrities to mislead Americans about Assad"", ""American taxpayer money i...",John Oliver is using celebrities to mislead Americans about Assad,15,14.0,0,2018-04-17 11:33:09+00:00,4630
3327,4631,1,6012,7,2001730,"[""John Oliver is using celebrities to mislead Americans about Assad"", ""American taxpayer money i...",American taxpayer money is funding Al Qaeda and ISIS,15,14.0,1,2018-04-17 11:33:09+00:00,4631
3328,4632,0,4352,5,2001730,"[""John Oliver is using celebrities to mislead Americans about Assad"", ""American taxpayer money i...",Obama is responsible for drone strikes killing innocents,15,14.0,0,2018-04-17 11:33:09+00:00,4632
3329,4633,0,6012,5,2001730,"[""John Oliver is using celebrities to mislead Americans about Assad"", ""American taxpayer money i...",Liberals are responsible for supporting harmful actions,15,14.0,0,2018-04-17 11:33:09+00:00,4633


In [114]:
YT_claim_embeds_exp_explode_join = YT_claim_embeds_exp_explode.merge(similarity_claim_exp.loc[:, ['claim_exp_index', 'most_similar_claim_index', 'similarity_score']], left_on='exp_explode_index', right_on='claim_exp_index', suffixes=('', '_exp_gpt4o'), how='left')

In [115]:
YT_claim_embeds_exp_explode_join.columns.to_list()

['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'commentText',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',


In [116]:
YT_claim_embeds_exp_explode_join.rename(columns={'most_similar_claim_index': 'most_similar_claim_index_exp_gpt4o', 'similarity_score': 'similarity_score_claim_exp_gpt4o'}, inplace=True)

In [120]:
YT_claim_embeds_exp_explode_join.loc[:, ['most_similar_claim_index_exp_gpt4o', 'exp_explode_index']].dtypes

most_similar_claim_index_exp_gpt4o    object
exp_explode_index                      int64
dtype: object

In [124]:
def convert_to_int(value):
    if isinstance(value, list):
        return np.nan
    try:
        return int(value)
    except ValueError:
        return np.nan

YT_claim_embeds_exp_explode_join.loc[: , 'most_similar_claim_index_exp_gpt4o_int'] = YT_claim_embeds_exp_explode_join['most_similar_claim_index_exp_gpt4o'].apply(convert_to_int)

In [128]:
YT_claim_embeds_exp_explode_join = YT_claim_embeds_exp_explode_join.merge(YT_claim_embeds_exp_explode.loc[:, ['claims_exp_split', 'exp_explode_index']], left_on='most_similar_claim_index_exp_gpt4o_int', right_on='exp_explode_index', suffixes=('', '_most_similar'), how='left')
YT_claim_embeds_exp_explode_join.drop(columns=['exp_explode_index_most_similar'], inplace=True)

In [ ]:
YT_claim_embeds_exp_explode_join.drop(columns=['most_similar_claim_index_exp_gpt4o'], inplace=True)
YT_claim_embeds_exp_explode_join.loc[:, 'similarity_score_claim_exp_gpt4o'] = YT_claim_embeds_exp_explode_join['similarity_score_claim_exp_gpt4o'].apply(convert_to_int)	

In [135]:
#save data:
YT_claim_embeds_exp_explode_join.to_parquet(f'{CFG.report_dir}/pubsphere_YT_post_claim_exp_explode.parquet')

In [7]:
#load exploded claims data:
YT_claim_embeds_exp_explode = pd.read_parquet(f'{CFG.report_dir}/pubsphere_YT_post_claim_exp_explode.parquet')
YT_claim_embeds_explode = pd.read_parquet(f'{CFG.report_dir}/pubsphere_YT_post_claim_explode.parquet')

In [14]:
#load packages and obtain the ADA embedding for expanded posts and original and expanded claims:
from tqdm import tqdm          #run these lines to run the ADA progress bar
tqdm.pandas()

In [60]:
dataset_claim_embeds.columns.to_list()

['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'commentText',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',


In [64]:
dataset_claim_embeds.groupby('Platform')['post_expansion_gpt4o'].nunique()

Platform
1    3119
2       0
Name: post_expansion_gpt4o, dtype: int64

In [65]:
dataset_claim_embeds.loc[~dataset_claim_embeds['post_expansion_gpt4o'].isna(), 'post_expansion_gpt4o']

0                                                                                                       sad
1       That's a vicious insult!!! What did a box of rocks ever do to you that you would slander it like...
3       Goya Solidar.  So there are a few of us left.  Try reading the comments on both Fox News and The...
4                                                                   hello hello \nNo-one else will hug him.
5       Please spare us you BS, Pocahantas! You will never be president of ths great country, daughter o...
                                                       ...                                                 
3854                                                                             Hints from evil to evil...
3855    If you were going to hire more than one person for this, then you should've hired the akatsuki c...
3859    Nah, they knew all about the cameras.  I'm guessing they look nothing like the people think.  Th...
3860    Alexander Hamilton. 

In [66]:
dataset_claim_embeds.loc[:, 'post_exp_gpt4o_ada_embedding'] = dataset_claim_embeds.loc[~dataset_claim_embeds['post_expansion_gpt4o'].isna(), 'post_expansion_gpt4o'].progress_apply(get_ada_embedding)

100%|██████████| 3132/3132 [13:39<00:00,  3.82it/s] 


In [71]:
#check whether the embedding is correctly joined to the dataset, since it is only run on a subset...:
dataset_claim_embeds.loc[:, ['post_expansion_gpt4o', 'post_exp_gpt4o_ada_embedding']]

,post_expansion_gpt4o,post_exp_gpt4o_ada_embedding
0,sad,"[0.0012491943780332804, -0.01312993373721838, 0.004941582679748535, -0.026987144723534584, -0.01..."
1,That's a vicious insult!!! What did a box of rocks ever do to you that you would slander it like...,"[-0.012995055876672268, -0.016615739092230797, 0.018477026373147964, -0.013490946963429451, -0.0..."
2,None,NaN
3,Goya Solidar. So there are a few of us left. Try reading the comments on both Fox News and The...,"[-0.0008296146406792104, 0.0013742600567638874, 0.010784990154206753, -0.030297795310616493, -0...."
4,hello hello \nNo-one else will hug him.,"[-0.026431242004036903, -0.003406326286494732, 0.005844608414918184, -0.01946660876274109, -0.00..."
...,...,...
3857,None,NaN
3858,None,NaN
3859,"Nah, they knew all about the cameras. I'm guessing they look nothing like the people think. Th...","[0.002804453717544675, -0.009028810076415539, 0.024697821587324142, -0.02455144375562668, 0.0020..."
3860,Alexander Hamilton. Troops are waiting in the fields for you. If you join us right now together ...,"[-0.027749337255954742, -0.024834491312503815, -0.00856317114084959, -0.01075902208685875, 0.006..."


In [ ]:
#appears correct

In [ ]:
YT_claim_embeds_exp_explode.loc[:, 'claim_exp_ada_embedding'] = YT_claim_embeds_exp_explode["claims_exp_split"].progress_apply(get_ada_embedding)

  0%|          | 0/6298 [00:00<?, ?it/s]

100%|██████████| 6298/6298 [28:57<00:00,  3.63it/s]  


In [ ]:
YT_claim_embeds_exp_explode


In [ ]:
#NOTE: we could test whether most similar claim is also the closest claim with respect to the ada embedding

In [19]:
#get ADA embeddings for posts:
YT_claim_embeds_explode.loc[:, 'claim_ada_embedding'] = YT_claim_embeds_explode['claims_split'].progress_apply(get_ada_embedding)

100%|██████████| 6211/6211 [28:33<00:00,  3.62it/s]  


In [21]:
#save data:
YT_claim_embeds_explode.to_parquet(f'{CFG.report_dir}/pubsphere_YT_post_claim_explode.parquet')
YT_claim_embeds_exp_explode.to_parquet(f'{CFG.report_dir}/pubsphere_YT_post_claim_exp_explode.parquet')

In [ ]:
#select all YT threads with 2+ comments, to ensure the possibility of a discussion and proper time labelling (-> YT) of comments to determine chronological order of comments:
YTdebatethreads = dataset_claim_embeds[dataset_claim_embeds['videoTitle'].isin(dataset_claim_embeds['videoTitle'].value_counts()[dataset_claim_embeds['videoTitle'].value_counts() >= 2].index)]
YTdebatethreads

In [59]:
#save data:
YTdebatethreads.to_parquet(f'{CFG.report_dir}/pubsphere_YT_debate_threads.parquet')

In [54]:
YTdebatethreads.videoTitle.value_counts()

videoTitle
'Hannity' panel on the important questions Mueller needs to answer                                23
Anderson Cooper lays out questions surrounding Mueller report                                     23
Sanders: Bolton is a guy who likes war                                                            22
Hannity: Trump puts Iran on notice after drone shot down                                          20
"Haley: Airstrikes ""crippled"" Syria’s chemical weapons program  "                               20
                                                                                                  ..
Andrew Yang Shares His Big Idea For Universal Basic Income | NBC Nightly News                      2
President Trump Signs Executive Order To Review National Monuments, Protected Lands | NBC News     2
Accidental Icon: 64 Year-Old Fordham Professor Becomes A Fashion Star | NBC Nightly News           2
Watch Live: Trump Delivers Remarks After El Paso And Dayton Weekend Shootings | 

In [55]:
YTdebatethreads.videoTitle.value_counts().mean()

np.float64(5.281115879828326)

In [ ]:
#the 466 videos contain on average 5.3 comments, with a minimum of 2 and a maximum of 23 comments
#they comprise 466 / 1137 = 41.0% of the videos, and 2461 / 3132 = 78.6% of the YT comments

In [37]:
dataset_claim_embeds.groupby(['Platform'])['videoTitle'].nunique()

Platform
1    1137
2       0
Name: videoTitle, dtype: int64

In [56]:
YTdebatethreads.columns.to_list()

['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'commentText',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',
